### **DADOS DO HOSPITAL SÍRIO-LIBANÊS (HSL)**

### Data: 19/10/2021

####Filipe Loyola Lopes


Informativo: 
- Análise dos dados com a classificação de gravidade a partir da origem do exame. 
- Origens dos exames: pronto socorro, internação ou UTI.
- Os pacientes foram divididos em quatro grupos e cada grupo classificado como GRAVE ou NÃO_GRAVE, conforme abaixo:

GRUPO_0 - pacientes com exames provindos apenas do pronto socorro (NÃO_GRAVE); 

GRUPO_1 - pacientes com exames provindos do pronto socorro e internação (NÃO_GRAVE);

GRUPO_2 - pacientes com exames provindos do pronto socorro e UTI (GRAVE).

GRUPO_3 - pacientes com exames provindos do pronto socorro, internação e UTI (GRAVE).





Links úteis:

https://www.vooo.pro/insights/12-tecnicas-pandas-uteis-em-python-para-manipulacao-de-dados/


https://medium.com/data-hackers/pandas-combinando-data-frames-com-merge-e-concat-10e7d07ca5ec

https://minerandodados.com.br/analise-de-dados-com-python-usando-pandas/


In [ ]:
#Bibliotecas

import numpy as np
import pandas as pd
from pandas import DataFrame
import csv
from numpy import mean
from numpy import std
from numpy import correlate
from numpy.random import randn
from numpy.random import seed
from matplotlib import pyplot
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
from google.colab import files
import datetime as dt
from matplotlib import pyplot as plt
plt.style.use('default')
#%matplotlib inline
import seaborn as sns
import warnings
import datetime as dt
from datetime import date

  ### **INTEGRAÇÃO DE DADOS**





















In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### **DATASET HSL_PACIENTES**

In [ ]:
# arquivo "HSL_Pacientes_3.csv" referente a janeiro 2021
pacientes = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/2021 dezembro Artigo/HSL_Pacientes_3.csv', sep='|')
print(pacientes.shape)
pacientes.head(3)

(8971, 7)


,ID_PACIENTE,IC_SEXO,aa_nascimento,CD_PAIS,CD_UF,CD_MUNICIPIO,CD_CEPREDUZIDO
0,3487791F44C34B421C932DC8616A8437,M,1963,BR,SP,MMMM,CCCC
1,0AD1FFA4419472256666A3445414F1F9,M,1969,BR,SP,MMMM,CCCC
2,4DECB3854B261CD1568A5F1800469797,M,1963,BR,SP,SAO PAULO,CCCC


In [ ]:
#verificando a existência de valores duplicatos
pacientes['ID_PACIENTE'].nunique()

8971

#### **DATASET HSL_EXAMES**

In [ ]:
# arquivo "HSL_Exames_3.csv" 
sirio_libanes = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/2021 dezembro Artigo/HSL_Exames_3.csv', sep='|')
print(sirio_libanes.shape)
sirio_libanes.head(2)

(1463834, 9)


,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,CD_UNIDADE,DE_VALOR_REFERENCIA
0,769E89D426E84A4797EF495608A0429E,261F2494FE76CC0F74C9A7A5A841C404,26/11/2020,Recepção do Centro Diagnóstico,17 Alfa Hidroxiprogesterona,17-Alfa-Hidroxiprog.,35,ng/dL,Ver resultado tradicional
1,9DECD77C85AD8EF58C52F5D747546632,F1C30774A4B98ECA55D2DF430DFA0025,06/10/2020,Recepção do Centro Diagnóstico,17 Hidroxipregnenolona,17-Hidroxi Pregnenolona,44,ng/dL,Ver resultado tradicional


In [ ]:
#Eliminando exemplos repetidos
sirio_libanes = sirio_libanes.drop_duplicates()
sirio_libanes.shape

(1436537, 9)

In [ ]:
#verificando analitos unicos 

sirio_libanes['DE_ANALITO'].nunique()

954

#### **DATASET HSL_DESFECHO**

In [ ]:
# arquivo "HSL_Desfechos_3.csv"
desfecho = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/2021 dezembro Artigo/HSL_Desfechos_3.csv', sep='|')
desfecho.shape

(42691, 8)

In [ ]:
#Eliminando exemplos repetidos
desfecho = desfecho.drop_duplicates()
desfecho.shape

(42691, 8)

In [ ]:
desfecho.head(3)

,ID_PACIENTE,ID_ATENDIMENTO,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,ID_CLINICA,DE_CLINICA,DT_DESFECHO,DE_DESFECHO
0,EAADD95C88D58DDAAA38953C8E49DD6C,03F7868BD8361515CC9AA4C7A175962A,30/11/2020,Ambulatorial,11,Consulta,30/11/2020,Alta a pedido
1,B5A04CD819AD85BDA8D688A5F7CAF4E5,FB5A9F180E5C1B7136E13D1C9C4F7A98,22/10/2020,Pronto Atendimento,42,CL Médica Síndromes Virais,22/10/2020,Alta a pedido
2,5E737D6235A9464E7D0E0AB21FCD4DA9,359CA9140E2A478FB7ED4D782F339BF9,21/07/2020,Pronto Atendimento,42,CL Médica Síndromes Virais,21/07/2020,Alta a pedido


In [ ]:
desfecho['ID_PACIENTE'].nunique()

8928

In [ ]:
desfecho['DE_DESFECHO'].value_counts()

Alta Administrativa                                                 29613
Alta médica melhorado                                               12277
Desistência do atendimento                                            229
Alta a pedido                                                         210
Alta médica Inalterado                                                145
Óbito após 48hs de internação sem necrópsia                            78
Alta médica curado                                                     61
Alta por abandono                                                      39
Transferência Inter-Hospitalar Externa - Serviço de Ambulância         15
Óbito nas primeiras 48hs de internação sem necrópsia não agônico       13
Assistência Domiciliar                                                  7
Transferência Inter-Hospitalar Externa - Transporte Próprio             2
Óbito após 48hs de internação com necrópsia                             1
Óbito nas primeiras 48hs de internação

In [ ]:
desfecho['DE_TIPO_ATENDIMENTO'].value_counts()

Ambulatorial          17977
Externo               11388
Pronto Atendimento    10894
Internado              2432
Name: DE_TIPO_ATENDIMENTO, dtype: int64

  #### **JUNTANDO DATASET'S**

  Exames e desfecho

In [ ]:
#adiciona colunas do dataset 'desfecho' na frente dos repectivos id_paciente e id_atendimento iguais
sirio = sirio_libanes.merge(desfecho, on = ["ID_PACIENTE", "ID_ATENDIMENTO"], how = "left")
sirio.head(3)

,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,CD_UNIDADE,DE_VALOR_REFERENCIA,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,ID_CLINICA,DE_CLINICA,DT_DESFECHO,DE_DESFECHO
0,769E89D426E84A4797EF495608A0429E,261F2494FE76CC0F74C9A7A5A841C404,26/11/2020,Recepção do Centro Diagnóstico,17 Alfa Hidroxiprogesterona,17-Alfa-Hidroxiprog.,35,ng/dL,Ver resultado tradicional,26/11/2020,Externo,20.0,Procedimentos,26/11/2020,Alta Administrativa
1,9DECD77C85AD8EF58C52F5D747546632,F1C30774A4B98ECA55D2DF430DFA0025,06/10/2020,Recepção do Centro Diagnóstico,17 Hidroxipregnenolona,17-Hidroxi Pregnenolona,44,ng/dL,Ver resultado tradicional,06/10/2020,Externo,20.0,Procedimentos,06/10/2020,Alta Administrativa
2,CE22DCFCE0BF5718B1781E4F01643661,53355F35A7D8DBE10777D1C66A58D05E,26/05/2020,Laboratório de Patologia Clínica,"Acido Ascorbico, plasma","Ácido ascorbico, plasma","0,8",mg/dL,"0,4 a 2,0",26/05/2020,Externo,20.0,Procedimentos,26/05/2020,Alta Administrativa


In [ ]:
sirio.shape

(1436537, 15)

In [ ]:
sirio.head(1)

,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,CD_UNIDADE,DE_VALOR_REFERENCIA,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,ID_CLINICA,DE_CLINICA,DT_DESFECHO,DE_DESFECHO
0,769E89D426E84A4797EF495608A0429E,261F2494FE76CC0F74C9A7A5A841C404,26/11/2020,Recepção do Centro Diagnóstico,17 Alfa Hidroxiprogesterona,17-Alfa-Hidroxiprog.,35,ng/dL,Ver resultado tradicional,26/11/2020,Externo,20.0,Procedimentos,26/11/2020,Alta Administrativa


In [ ]:
pacientes.head(1)

,ID_PACIENTE,IC_SEXO,aa_nascimento,CD_PAIS,CD_UF,CD_MUNICIPIO,CD_CEPREDUZIDO
0,3487791F44C34B421C932DC8616A8437,M,1963,BR,SP,MMMM,CCCC


https://medium.com/data-hackers/pandas-combinando-data-frames-com-merge-e-concat-10e7d07ca5ec



Obtendo SEXO e Ano de nascimento da planilha HSL_PACIENTES

In [ ]:
pacientes_2 = pacientes[['ID_PACIENTE','aa_nascimento','IC_SEXO']]

pacientes_2.head(1)

,ID_PACIENTE,aa_nascimento,IC_SEXO
0,3487791F44C34B421C932DC8616A8437,1963,M


#### **DATASET SIRIO**

In [ ]:
#adiciona a coluna aa_nacimento do dataframe pacientes em sirio
sirio = sirio.merge(pacientes_2, on=['ID_PACIENTE'], how='left')
sirio.head(3)

,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,CD_UNIDADE,DE_VALOR_REFERENCIA,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,ID_CLINICA,DE_CLINICA,DT_DESFECHO,DE_DESFECHO,aa_nascimento,IC_SEXO
0,769E89D426E84A4797EF495608A0429E,261F2494FE76CC0F74C9A7A5A841C404,26/11/2020,Recepção do Centro Diagnóstico,17 Alfa Hidroxiprogesterona,17-Alfa-Hidroxiprog.,35,ng/dL,Ver resultado tradicional,26/11/2020,Externo,20.0,Procedimentos,26/11/2020,Alta Administrativa,1974,F
1,9DECD77C85AD8EF58C52F5D747546632,F1C30774A4B98ECA55D2DF430DFA0025,06/10/2020,Recepção do Centro Diagnóstico,17 Hidroxipregnenolona,17-Hidroxi Pregnenolona,44,ng/dL,Ver resultado tradicional,06/10/2020,Externo,20.0,Procedimentos,06/10/2020,Alta Administrativa,1982,F
2,CE22DCFCE0BF5718B1781E4F01643661,53355F35A7D8DBE10777D1C66A58D05E,26/05/2020,Laboratório de Patologia Clínica,"Acido Ascorbico, plasma","Ácido ascorbico, plasma","0,8",mg/dL,"0,4 a 2,0",26/05/2020,Externo,20.0,Procedimentos,26/05/2020,Alta Administrativa,1984,F


In [ ]:
sirio.shape

(1436537, 17)

In [ ]:
sirio['ID_PACIENTE'].nunique()

8971

---

#### **VERIFICANDO VALORES NULOS**

In [ ]:
# cópia profunda do dataframe para não alterar o df original
sirio2 = sirio.copy(deep=True)

# Considera apenas o último exame no caso de repetidos no mesmo dia.
sirio2 = sirio2.groupby(['ID_PACIENTE', 'ID_ATENDIMENTO','DT_COLETA','DE_ANALITO']).agg({'DE_RESULTADO' : ['last']}).reset_index()

# solucionando nome colunas dois niveis
sirio2.columns = [ '_'.join(x) for x in sirio2.columns ]

# criando chave a partir de ID_PACIENTE, ID_ATENDIMENTO e DT_COLETA 
sirio2['chave'] = sirio2['ID_PACIENTE_']+'.'+sirio2['ID_ATENDIMENTO_']+'.'+sirio2['DT_COLETA_']

sirio2 = sirio2[['chave','DE_ANALITO_','DE_RESULTADO_last']]

sirio2.columns = ['chave', 'analito','resultado']

print(sirio2.shape)

sirio2.head(3)

(1288263, 3)


,chave,analito,resultado
0,000150DB429AFF026ECD130B0A076CEF.644998DCF29A2...,Coronavírus (2019-nCoV),DETECTADO (POSITIVO)
1,000150DB429AFF026ECD130B0A076CEF.644998DCF29A2...,Material (2019-nCoV),raspado de material do trato respiratório
2,00017961865C4F766FDBB3CD8FE0BFB0.B57529234B613...,ALT (TGP),26


In [ ]:
sirio_pivot = sirio2.pivot(index='chave', 
    
                    columns='analito',
                                      
                    values='resultado').reset_index()

print(sirio_pivot.shape)

sirio_pivot.head()

(39104, 955)


analito,chave,17-Alfa-Hidroxiprog.,17-Hidroxi Pregnenolona,183,25OH-Vitamina D Total,5-Nucleotidase,ACTH,ADA,ALT (TGP),ANCA,...,"Ácido ascorbico, plasma",Ácido aspártico,Ácido fólico,Ácido glutâmico,"Ácido metilmalonico, soro",Ácido Úrico,"Ácido Úrico, urina",Ácidos biliares totais,Ésteres de Carnitina (EC),Índice de Green & King
0,000150DB429AFF026ECD130B0A076CEF.644998DCF29A2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00017961865C4F766FDBB3CD8FE0BFB0.B57529234B613...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000F0BC139D2846DB86AA32B8F05B215.9D6B251913C9B...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,000F0BC139D2846DB86AA32B8F05B215.C8CD5BD3F5BAF...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"7,8",NaN,NaN,NaN,NaN
4,00177290279939FB33386B29198C450E.94B6DCEC9250D...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#sirio_pivot.to_csv('sirio_pivot.csv', sep='|', encoding='utf-8') # gera csv co

In [ ]:
valores_nulos = pd.DataFrame()

valores_nulos['Null'] = sirio_pivot.isnull().sum()

valores_nulos = valores_nulos.reset_index()

#valores_nulos = valores_nulos.T

# Get names of indexes for which column Stock has value No
indexNames = valores_nulos[ valores_nulos['analito'] == 'chave' ].index

# Delete these row indexes from dataFrame
valores_nulos.drop(indexNames , inplace=True)

print(valores_nulos.head())

print("\nshape: ", valores_nulos.shape, "\n")

valores_nulos.describe()

                   analito   Null
1     17-Alfa-Hidroxiprog.  39062
2  17-Hidroxi Pregnenolona  39099
3                      183  37035
4    25OH-Vitamina D Total  37497
5           5-Nucleotidase  39101

shape:  (954, 2) 



,Null
count,954.000000
mean,37753.619497
std,4836.900438
min,11354.000000
25%,38920.500000
50%,39086.500000
75%,39100.000000
max,39103.000000


In [ ]:
# No total são 39104 exemplos
# Transformando valor absoluto em porcentagem:

# Media
media_null = (37753.619497 / 39104)*100
print("Média: ", media_null)

# desvio padrão
desvio_null = (4836.900438 / 39104)*100
print("\nDesvio padrão: ", desvio_null)

# mínimo
minimo_null = (11354 / 39104)*100
print("\nMínimo: ", minimo_null)

# máximo
maximo_null = (39103 / 39104)*100
print("\nMáximo: ", maximo_null)


Média:  96.5466947038666

Desvio padrão:  12.36932395151391

Mínimo:  29.035392798690673

Máximo:  99.9974427168576




---

#### **FILTRO 1: SELEÇÃO MANUAL DE ATRIBUTOS**





In [ ]:
# Excluindo colunas desnecessárias
df_sirio = sirio.drop(columns=['CD_UNIDADE', 'DE_VALOR_REFERENCIA'])
print(df_sirio.shape)
df_sirio.head(3)   

(1436537, 15)


,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,ID_CLINICA,DE_CLINICA,DT_DESFECHO,DE_DESFECHO,aa_nascimento,IC_SEXO
0,769E89D426E84A4797EF495608A0429E,261F2494FE76CC0F74C9A7A5A841C404,26/11/2020,Recepção do Centro Diagnóstico,17 Alfa Hidroxiprogesterona,17-Alfa-Hidroxiprog.,35,26/11/2020,Externo,20.0,Procedimentos,26/11/2020,Alta Administrativa,1974,F
1,9DECD77C85AD8EF58C52F5D747546632,F1C30774A4B98ECA55D2DF430DFA0025,06/10/2020,Recepção do Centro Diagnóstico,17 Hidroxipregnenolona,17-Hidroxi Pregnenolona,44,06/10/2020,Externo,20.0,Procedimentos,06/10/2020,Alta Administrativa,1982,F
2,CE22DCFCE0BF5718B1781E4F01643661,53355F35A7D8DBE10777D1C66A58D05E,26/05/2020,Laboratório de Patologia Clínica,"Acido Ascorbico, plasma","Ácido ascorbico, plasma","0,8",26/05/2020,Externo,20.0,Procedimentos,26/05/2020,Alta Administrativa,1984,F


#### DATETIME

In [ ]:
#Formato data
df_sirio['DT_ATENDIMENTO'] = pd.to_datetime(df_sirio['DT_ATENDIMENTO'])
df_sirio['DT_COLETA'] = pd.to_datetime(df_sirio['DT_COLETA'])
df_sirio['DT_DESFECHO'] = pd.to_datetime(df_sirio['DT_COLETA'])

#### **FILTRO 2: VERIFICANDO PACIENTES COM COVID POSITIVO**

In [ ]:
tipos_exames = df_sirio['DE_EXAME'].value_counts()
print(tipos_exames.shape)
tipos_exames

(665,)


Hemograma                                                     638393
Calculo da Estimativa da Taxa, de Filtracao Glomular, soro    121404
Gasometria Venosa                                              67068
Gasometria Arterial                                            51936
Urina Tipo I - Jato Medio                                      44064
                                                               ...  
Coagulograma, especial, sangue total                               1
Zika vírus, detecção do RNA por PCR em tempo real, plasma          1
Rast Peixe (Bacalhau) F3                                           1
3 Alfa Diol G                                                      1
Clobazam                                                           1
Name: DE_EXAME, Length: 665, dtype: int64

In [ ]:
tipos_exames.to_csv('tipos_exames.csv', sep='|', encoding='utf-8') # gera csv com os tipos de exame

In [ ]:
#**Lista de exames de COVID**
#Analisando os tipos de exames foram identificados aqueles que são teste de COVID-19 (ls_exames_covid)

ls_exames_covid = ['COVID-19-PCR para SARS-COV-2, Vários Materiais (Fleury)', 
                   'COVID-19-Sorologia IgM e IgG por quimiluminescência, soro', 
                   'Detecção de Coronavírus (NCoV-2019) POR PCR (Anatomia Patológica)',
                   'COVID-19-Teste Rápido (IgM e IgG), soro', 
                   'COVID-19, anticorpos IGA e IGG, soro', 
                   'Sars Cov-2, Teste Molecular Rápido Para Detecção, Vários Materiais', 
                   'Sorologia - Coronavírus, IgG', 
                   'Sorologia - Coronavírus, IgA']

In [ ]:
df_exames_covid = df_sirio.loc[df_sirio['DE_EXAME'].isin(ls_exames_covid)]
#df_exames_covid.to_csv('exames_covid', sep='\t', encoding='utf-8')
df_exames_covid.shape

(23654, 15)

In [ ]:
# Tipos de resultados para o exame de covid
resultados_covid = df_exames_covid['DE_RESULTADO'].value_counts()
#resultados_covid.to_csv('tipos_resultados.csv', sep='\t', encoding='utf-8')
resultados_covid.shape

(599,)

In [ ]:
# Foi realizada uma análise para identificar os resultados que indicam COVID POSITIVOS (ls_resultados_positivos)

ls_resultados_positivos = ['DETECTADO',
                           'DETECTADO (POSITIVO)',
                           'REAGENTE',
                           'Detectados anticorpos da classe IgG contra SARS-CoV-2, Este perfil é compatível com infecção pregressa, Estudos demonstram que, sobretudo em pessoas que apresentaram quadro clínico leve ou não apresentaram sintomas, os níveis de anticorpos podem diminuir ao longo do tempo, podendo inclusive, se tornar indetectáveis (negativos), O papel destes anticorpos na proteção contra reinfecção não é completamente estabelecido,',
                           'Amostra REAGENTE para IgG contra SARS-CoV-2,',
                           'Detectados anticorpos das classes IgM e IgG contra SARS-CoV-2, Este perfil sugere infecção recente, Estudos demonstram que, sobretudo em pessoas que apresentaram quadro clínico leve ou não apresentaram sintomas, os níveis de anticorpos podem diminuir ao longo do tempo, podendo inclusive, se tornar indetectáveis (negativos), O papel destes anticorpos na proteção contra reinfecção não é completamente estabelecido,',
                           'Amostra REAGENTE para IgM e IgG contra SARS-CoV-2,',
                           'Evidência sorológica de infecção recente por SARS-CoV-2,',
                           'Detectados anticorpos da classe IgM contra SARS-CoV-2, Este perfil é compatível com soroconversão inicial ou produção de baixos níveis de anticorpos da classe IgG, Estudos demonstram que, sobretudo em pessoas que apresentaram quadro clínico leve ou não apresentaram sintomas, a soroconversão pode ocorrer mais tardiamente, em baixos níveis de anticorpos, ou mesmo não ocorrer, Sugere-se seguimento sorológico para avaliar a soroconversão de IgG em, no mínimo, 7 dias,',
                           'Evidência sorológica de infecção pregressa por SARS-CoV-2,',
                           'Detectados anticorpos totais contra SARS-CoV-2, porém não foi possível definir, nesta amostra, a(s) classe(s) de imunoglobulina(s) presente(s) (IgM e/ou IgG), Estudos demonstram que, sobretudo em pessoas que apresentaram quadro clínico leve ou não apresentaram sintomas, a soroconversão pode ocorrer mais tardiamente ou em baixos níveis de anticorpos, A aparente discrepância observada entre as metodologias pode se dever à diferença de sensibilidade ou à utilização de antígenos distintos, Sugere-se o seguimento sorológico em, no mínimo, 7 dias,',
                           'Amostra REAGENTE para anticorpos contra SARS-CoV-2,',
                           'Amostra REAGENTE para IgM contra SARS-CoV-2,',
                           'Detectados anticorpos da classe IgM contra SARS-CoV-2, em apenas uma das metodologias utilizadas, A possibilidade de falsa reatividade não pode ser descartada, Sugere-se seguimento sorológico em, no mínimo, 7 dias,',
                           'Amostra REAGENTE para IgG contra SARS-CoV-2, em apenas uma metodologia,',
                           'Amostra REAGENTE para IgM contra SARS-CoV-2, em apenas uma metodologia,',
                           'Detectados anticorpos da classe IgG contra SARS-CoV-2 em baixos níveis, em apenas uma das metodologias utilizadas, A possibilidade de falsa reatividade não pode ser descartada, embora a aparente discrepância entre as metodologias possa se dever às diferenças de sensibilidade  ou à utilização de antígenos distintos,  Sugere-se seguimento sorológico em, no mínimo, 7 dias,',
                           'Possível evidência sorológica de infecção recente por SARS-CoV-2,',
                           'Detectados anticorpos da classe IgG contra SARS-CoV-2 em baixos níveis, em apenas uma das metodologias utilizadas, Este perfil pode se dever à soroconversão inicial ou à produção de baixos níveis de anticorpos, contudo, a possibilidade de falsa reatividade não pode ser descartada, Sugere-se seguimento sorológico em, no mínimo, 7 dias,',
                           'Detectados anticorpos da classe IgG contra SARS-CoV-2, Este perfil é compatível com infecção pregressa, Estudos mostram que, sobretudo em pessoas que apresentaram quadro clínico leve ou não apresentaram sintomas, os níveis de anticorpos podem diminuir ao longo do tempo, podendo inclusive, tornar-se negativos, O papel destes anticorpos na proteção contra reinfecção não é completamente estabelecido,',
                           'Possível evidência sorológica de infecção recente por SARS-COV-2,',
                           'O resultado sugere que já tenham transcorrido mais de 3 semanas da infecção aguda, A capacidade protetora dos anticorpos da classe IgG não é completamente estabelecida,;'
                           ]

In [ ]:
df_covid_positivo = df_exames_covid.loc[df_exames_covid['DE_RESULTADO'].isin(ls_resultados_positivos)]

df_covid_positivo.shape

(11299, 15)

In [ ]:
#número de pacientes com covid positivo
df_covid_positivo['ID_PACIENTE'].nunique()

8904

In [ ]:
#pacientes com covid positivo
pacientes_positivos = df_covid_positivo['ID_PACIENTE'].unique()

pacientes_positivos

array(['40A4A8FC3EBB00EE82E926426A0B7144',
       'BD7D6D9504C0B9B7EA7B6F7D84100736',
       'E6460EA0BDC9395695513A4981646113', ...,
       'A996DC50BF9BB5A257A63840DC5B6140',
       '1EA10A9B725F4824E151F520C9506928',
       '956CD04CE01946A55895E53D1DE1A086'], dtype=object)

In [ ]:
df_covid_positivo = df_sirio.loc[df_sirio['ID_PACIENTE'].isin(pacientes_positivos)]

print(df_covid_positivo.shape)

df_covid_positivo['ID_PACIENTE'].nunique()

(1427654, 15)


8904

In [ ]:
#pacientes com covid negativo
df_covid_negativo = df_sirio.loc[~df_sirio['ID_PACIENTE'].isin(pacientes_positivos)]

print(df_covid_negativo.shape)

print(df_covid_negativo['ID_PACIENTE'].nunique())

(8883, 15)
67


In [ ]:
#verificando qual a porcentagem de pacientes para cada desfecho, entre os que não tiveram covid:
desfechos_pacientes_sem_covid = df_covid_negativo['DE_DESFECHO'].value_counts()

desfechos_pacientes_sem_covid

Alta médica melhorado                          5907
Alta Administrativa                            1897
Alta médica curado                              578
Óbito após 48hs de internação sem necrópsia     238
Alta médica Inalterado                            2
Name: DE_DESFECHO, dtype: int64

In [ ]:
df_covid_negativo.loc[df_covid_negativo['DE_DESFECHO'] =='Óbito após 48hs de internação sem necrópsia'].value_counts()

ID_PACIENTE                       ID_ATENDIMENTO                    DT_COLETA   DE_ORIGEM               DE_EXAME                          DE_ANALITO            DE_RESULTADO  DT_ATENDIMENTO  DE_TIPO_ATENDIMENTO  ID_CLINICA  DE_CLINICA      DT_DESFECHO  DE_DESFECHO                                  aa_nascimento  IC_SEXO
D1CFB8DFAAFAA11CC2D6A1B187616206  708F162F3FEF237B9697459B29224343  2020-08-06  Pronto Socorro          Bilirrubinas Total E Fracoes      Bilirrubina Direta    0,57          2020-08-06      Internado            1.0         Clínica Médica  2020-08-06   Óbito após 48hs de internação sem necrópsia  1938           M          1
                                                                    2020-11-06  Unidades de Internação  Creatinina                        Creatinina            3,33          2020-08-06      Internado            1.0         Clínica Médica  2020-11-06   Óbito após 48hs de internação sem necrópsia  1938           M          1
                              

In [ ]:
# PACIENTES POSITIVOS

df_hsl = df_covid_positivo

df_hsl['DE_ORIGEM'].value_counts()

UTI                                     695237
Unidades de Internação                  355123
Pronto Socorro                          145112
Recepção do Centro Diagnóstico          119802
Centro de Oncologia                      16412
                                         ...  
Saúde Corporativa - Banco Votorantim         2
Secretária do Centro Diagnóstico             2
Capacidade e Operações                       1
Unidade Cardiológica do Exercício            1
Anatomia Patológica - Jardins                1
Name: DE_ORIGEM, Length: 95, dtype: int64

In [ ]:
df_hsl.head(3)

,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,ID_CLINICA,DE_CLINICA,DT_DESFECHO,DE_DESFECHO,aa_nascimento,IC_SEXO
0,769E89D426E84A4797EF495608A0429E,261F2494FE76CC0F74C9A7A5A841C404,2020-11-26,Recepção do Centro Diagnóstico,17 Alfa Hidroxiprogesterona,17-Alfa-Hidroxiprog.,35,2020-11-26,Externo,20.0,Procedimentos,2020-11-26,Alta Administrativa,1974,F
1,9DECD77C85AD8EF58C52F5D747546632,F1C30774A4B98ECA55D2DF430DFA0025,2020-06-10,Recepção do Centro Diagnóstico,17 Hidroxipregnenolona,17-Hidroxi Pregnenolona,44,2020-06-10,Externo,20.0,Procedimentos,2020-06-10,Alta Administrativa,1982,F
2,CE22DCFCE0BF5718B1781E4F01643661,53355F35A7D8DBE10777D1C66A58D05E,2020-05-26,Laboratório de Patologia Clínica,"Acido Ascorbico, plasma","Ácido ascorbico, plasma","0,8",2020-05-26,Externo,20.0,Procedimentos,2020-05-26,Alta Administrativa,1984,F


In [ ]:
#agrupando pacientes para verificar última data de atendimento, para pacientes com mais de uma data de atendimento
atendimento_paciente = df_hsl.groupby(['ID_PACIENTE']).agg({'DT_ATENDIMENTO': ['max']}).reset_index()
atendimento_paciente.columns=['ID_PACIENTE', 'DT_ATENDIMENTO_MAXIMA']
atendimento_paciente['DT_ATENDIMENTO_MAXIMA'] = pd.to_datetime(atendimento_paciente['DT_ATENDIMENTO_MAXIMA'])
atendimento_paciente.head(3)

,ID_PACIENTE,DT_ATENDIMENTO_MAXIMA
0,000150DB429AFF026ECD130B0A076CEF,2020-03-12
1,00017961865C4F766FDBB3CD8FE0BFB0,2020-08-25
2,000F0BC139D2846DB86AA32B8F05B215,2020-09-24


In [ ]:
#juntando datasets para auxiliar próximo filtro que é Exame covid até 15 dias após DT_ATENDIMENTO_MAXIMA

df_hsl_temp = df_hsl.merge(atendimento_paciente, on=['ID_PACIENTE'], how='left')
df_hsl_temp.head(3)

,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,ID_CLINICA,DE_CLINICA,DT_DESFECHO,DE_DESFECHO,aa_nascimento,IC_SEXO,DT_ATENDIMENTO_MAXIMA
0,769E89D426E84A4797EF495608A0429E,261F2494FE76CC0F74C9A7A5A841C404,2020-11-26,Recepção do Centro Diagnóstico,17 Alfa Hidroxiprogesterona,17-Alfa-Hidroxiprog.,35,2020-11-26,Externo,20.0,Procedimentos,2020-11-26,Alta Administrativa,1974,F,2020-11-26
1,9DECD77C85AD8EF58C52F5D747546632,F1C30774A4B98ECA55D2DF430DFA0025,2020-06-10,Recepção do Centro Diagnóstico,17 Hidroxipregnenolona,17-Hidroxi Pregnenolona,44,2020-06-10,Externo,20.0,Procedimentos,2020-06-10,Alta Administrativa,1982,F,2020-10-09
2,CE22DCFCE0BF5718B1781E4F01643661,53355F35A7D8DBE10777D1C66A58D05E,2020-05-26,Laboratório de Patologia Clínica,"Acido Ascorbico, plasma","Ácido ascorbico, plasma","0,8",2020-05-26,Externo,20.0,Procedimentos,2020-05-26,Alta Administrativa,1984,F,2020-07-21


In [ ]:
#verificando se existe diferença entre DT_COLETA e DT_COLETA_MAXIMA
df_hsl_temp['delta_dt_atendimento'] = (df_hsl_temp['DT_ATENDIMENTO_MAXIMA']-df_hsl_temp['DT_ATENDIMENTO']).dt.days
df_hsl_temp.head(3)

,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,ID_CLINICA,DE_CLINICA,DT_DESFECHO,DE_DESFECHO,aa_nascimento,IC_SEXO,DT_ATENDIMENTO_MAXIMA,delta_dt_atendimento
0,769E89D426E84A4797EF495608A0429E,261F2494FE76CC0F74C9A7A5A841C404,2020-11-26,Recepção do Centro Diagnóstico,17 Alfa Hidroxiprogesterona,17-Alfa-Hidroxiprog.,35,2020-11-26,Externo,20.0,Procedimentos,2020-11-26,Alta Administrativa,1974,F,2020-11-26,0.0
1,9DECD77C85AD8EF58C52F5D747546632,F1C30774A4B98ECA55D2DF430DFA0025,2020-06-10,Recepção do Centro Diagnóstico,17 Hidroxipregnenolona,17-Hidroxi Pregnenolona,44,2020-06-10,Externo,20.0,Procedimentos,2020-06-10,Alta Administrativa,1982,F,2020-10-09,121.0
2,CE22DCFCE0BF5718B1781E4F01643661,53355F35A7D8DBE10777D1C66A58D05E,2020-05-26,Laboratório de Patologia Clínica,"Acido Ascorbico, plasma","Ácido ascorbico, plasma","0,8",2020-05-26,Externo,20.0,Procedimentos,2020-05-26,Alta Administrativa,1984,F,2020-07-21,56.0


In [ ]:
# Filtrando apenas pacientes com COVID confirmado até 15 dias após o atendimento

df_hsl_temp['delta_covid_positivo'] = (df_hsl_temp['DT_COLETA'] - df_hsl_temp['DT_ATENDIMENTO_MAXIMA']).dt.days

print(df_hsl_temp['delta_covid_positivo'].value_counts())

 0.0      198508
 1.0       29913
 2.0       18917
 3.0       16555
 4.0       15415
           ...  
 306.0        34
 210.0        26
 290.0        23
-359.0         6
 291.0         1
Name: delta_covid_positivo, Length: 645, dtype: int64


In [ ]:
#delta_covid_positivo = df_hsl_temp['delta_covid_positivo'].value_counts()
#delta_covid_positivo.to_csv('Delta_covid_positivo.csv', sep='|', encoding='utf8')

In [ ]:
#Selecionando apenas exemplos com delta_covid_positivo <= 15 para selecionar esses pacientes
quinze_dias = df_hsl_temp[df_hsl_temp['delta_covid_positivo']<=15]
quinze_dias = quinze_dias[quinze_dias['delta_covid_positivo']>=0]
print(quinze_dias.shape)
quinze_dias['delta_covid_positivo'].value_counts()

(363871, 18)


0.0     198508
1.0      29913
2.0      18917
3.0      16555
4.0      15415
5.0      14065
6.0      11873
7.0      10473
8.0       8658
9.0       8148
10.0      7502
11.0      6000
12.0      4976
13.0      4403
14.0      4393
15.0      4072
Name: delta_covid_positivo, dtype: int64

In [ ]:
pacientes_quinze_dias = quinze_dias['ID_PACIENTE'].unique()
print(pacientes_quinze_dias.shape)
pacientes_quinze_dias

(8750,)


array(['769E89D426E84A4797EF495608A0429E',
       '420FDC7BF5E9C3CA3725F1EB6B1DFC9D',
       '0375A54F4BAD09CEA28A86FEB1FD37AD', ...,
       'FDF970237573A9A2909057877AE0E37C',
       '3ECDA6E3F39A549ECB1AFCCEAEF5D3E0',
       '9EA293E956C62058B8AF93EA69D7FE85'], dtype=object)

In [ ]:
# Finalmente base com o filtro de 15 dias (covid positivo)

df_hsl = df_hsl.loc[df_hsl['ID_PACIENTE'].isin(pacientes_quinze_dias)]

print(df_hsl['ID_PACIENTE'].nunique())

df_hsl.head(3)

8750


,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,ID_CLINICA,DE_CLINICA,DT_DESFECHO,DE_DESFECHO,aa_nascimento,IC_SEXO
0,769E89D426E84A4797EF495608A0429E,261F2494FE76CC0F74C9A7A5A841C404,2020-11-26,Recepção do Centro Diagnóstico,17 Alfa Hidroxiprogesterona,17-Alfa-Hidroxiprog.,35,2020-11-26,Externo,20.0,Procedimentos,2020-11-26,Alta Administrativa,1974,F
1,9DECD77C85AD8EF58C52F5D747546632,F1C30774A4B98ECA55D2DF430DFA0025,2020-06-10,Recepção do Centro Diagnóstico,17 Hidroxipregnenolona,17-Hidroxi Pregnenolona,44,2020-06-10,Externo,20.0,Procedimentos,2020-06-10,Alta Administrativa,1982,F
2,CE22DCFCE0BF5718B1781E4F01643661,53355F35A7D8DBE10777D1C66A58D05E,2020-05-26,Laboratório de Patologia Clínica,"Acido Ascorbico, plasma","Ácido ascorbico, plasma","0,8",2020-05-26,Externo,20.0,Procedimentos,2020-05-26,Alta Administrativa,1984,F


#### **ANALISANDO AS ORIGENS DOS EXAMES PARA DIVISÃO EM GRUPOS**

GRUPO_0 - pacientes com exames provindos apenas do pronto socorro (NÃO_GRAVE);

GRUPO_1 - pacientes com exames provindos apenas do pronto socorro e internação (NÃO_GRAVE);

GRUPO_2 - pacientes com exames provindos do PS e UTI (GRAVE).

GRUPO_3 - pacientes com exames provindos do PS, Internação e UTI (GRAVE).

In [ ]:
# Origens categorizadas em três tipos: UTI (Unidade de Terapia Intensiva), INT (Internação) ou PS (Pronto Socorro)

grupos = df_hsl

grupos['UTI'] = 'NaN'  #Cria coluna UTI com valores nulos

grupos['INT'] = 'Nan'  #Cria coluna INT com valores nulos

grupos['PS'] = 'Nan'   #Cria coluna PS com valores nulos

print(grupos.shape)

grupos.head(2)

(1318899, 18)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,ID_CLINICA,DE_CLINICA,DT_DESFECHO,DE_DESFECHO,aa_nascimento,IC_SEXO,UTI,INT,PS
0,769E89D426E84A4797EF495608A0429E,261F2494FE76CC0F74C9A7A5A841C404,2020-11-26,Recepção do Centro Diagnóstico,17 Alfa Hidroxiprogesterona,17-Alfa-Hidroxiprog.,35,2020-11-26,Externo,20.0,Procedimentos,2020-11-26,Alta Administrativa,1974,F,NaN,Nan,Nan
1,9DECD77C85AD8EF58C52F5D747546632,F1C30774A4B98ECA55D2DF430DFA0025,2020-06-10,Recepção do Centro Diagnóstico,17 Hidroxipregnenolona,17-Hidroxi Pregnenolona,44,2020-06-10,Externo,20.0,Procedimentos,2020-06-10,Alta Administrativa,1982,F,NaN,Nan,Nan


In [ ]:
# Rotular exemplos provindos de UTI
# Toda origem "UTI" foi categorizada como UTI, tendo os atributos: 
# UTI = 1
# INT = 0
# PS = 0

exemplos_UTI = grupos[grupos['DE_ORIGEM']=='UTI']

print('total de exemplos UTI: ', exemplos_UTI.shape)

atendimentos_UTI = exemplos_UTI['ID_ATENDIMENTO'].unique()

print('Total de chaves unicas de atendimentos com exames UTI: ', atendimentos_UTI.shape)
print('\n')
print('\n')

exemplos_UTI['UTI'] = 1

exemplos_UTI['INT'] = 0

exemplos_UTI['PS'] = 0

exemplos_UTI.head(3)

total de exemplos UTI:  (600358, 18)
Total de chaves unicas de atendimentos com exames UTI:  (930,)






/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,ID_CLINICA,DE_CLINICA,DT_DESFECHO,DE_DESFECHO,aa_nascimento,IC_SEXO,UTI,INT,PS
109,CE30E08E37887C8B936E24D36D9CDE85,ADE8CE496E52B77B70699907940F739E,2020-09-26,UTI,Acido Fólico,Ácido fólico,"3,1",2020-08-26,Internado,1.0,Clínica Médica,2020-09-26,Óbito nas primeiras 48hs de internação sem nec...,1943,M,1,0,0
110,70ECA50821E10D1B747212B96A9FADB9,91D503356F2073546A847A9F8ABCF674,2020-07-07,UTI,Acido Fólico,Ácido fólico,"9,9",2020-05-07,Internado,1.0,Clínica Médica,2020-07-07,Alta médica melhorado,1951,F,1,0,0
111,594C2211DFCAC372C907D4D891520631,929ADD1F5DDE3D8217A14444221F3157,2020-07-20,UTI,Acido Fólico,Ácido fólico,"10,0",2020-06-07,Internado,1.0,Clínica Médica,2020-07-20,Óbito após 48hs de internação sem necrópsia,1947,M,1,0,0


In [ ]:
# Rotular exemplos provindos da INT
# Toda origem "Unidades de Internação" ou "Atendimento - Recepção Internação" foi categorizada como INT, ou seja: 
# UTI = 0
# INT = 1
# PS = 0

exemplos_INT = grupos.query('DE_ORIGEM == "Unidades de Internação" | DE_ORIGEM == "Atendimento - Recepção / Internação"')

print('total de exemplos INT: ', exemplos_INT.shape)

atendimentos_INT = exemplos_INT['ID_ATENDIMENTO'].unique()

print('Total de chaves unicas de atendimentos com exames em INT: ', atendimentos_INT.shape)

exemplos_INT['UTI'] = 0

exemplos_INT['INT'] = 1

exemplos_INT['PS'] = 0

exemplos_INT.head(3)


total de exemplos INT:  (345799, 18)
Total de chaves unicas de atendimentos com exames em INT:  (1848,)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,ID_CLINICA,DE_CLINICA,DT_DESFECHO,DE_DESFECHO,aa_nascimento,IC_SEXO,UTI,INT,PS
75,369C7BA557BDD9BD5A50EA5059A6C012,6A53F7CF2700AC25AE276DB792A21E25,2020-12-16,Unidades de Internação,Acido Fólico,Ácido fólico,"4,5",2020-12-11,Internado,1.0,Clínica Médica,2020-12-16,Alta médica melhorado,1935,M,0,1,0
76,D4D3CF95BAD0955F0123D95E1ABC124C,A4C48142455EB43F3FD280056ED0B163,2020-11-11,Unidades de Internação,Acido Fólico,Ácido fólico,"7,1",2020-02-10,Internado,8.0,Ambulância,2020-11-11,Alta médica melhorado,1940,M,0,1,0
77,8A631A521A1CF1AA34FEB5C36A5AEC88,4B944C8A29620042866167F115E8235C,2020-08-08,Unidades de Internação,Acido Fólico,Ácido fólico,"7,4",2020-06-21,Internado,1.0,Clínica Médica,2020-08-08,Alta médica melhorado,1946,M,0,1,0


Todos os outros tipos de origens foram categorizadas como PS, tendo os atributos: 

UTI = 0

INT = 0

PS = 1

In [ ]:
#Rotular exemplos provindos do PS

exemplos_PS = grupos.query('DE_ORIGEM != "Unidades de Internação" & DE_ORIGEM != "Atendimento - Recepção / Internação" & DE_ORIGEM != "UTI"')

print('total de exemplos PS: ', exemplos_PS.shape)

atendimentos_PS = exemplos_PS['ID_ATENDIMENTO'].unique()

print('Total de chaves unicas de atendimentos com exame em PS: ', atendimentos_PS.shape)

exemplos_PS['UTI'] = 0

exemplos_PS['INT'] = 0

exemplos_PS['PS'] = 1

exemplos_PS.head(5)


total de exemplos PS:  (372742, 18)
Total de chaves unicas de atendimentos com exame em PS:  (17302,)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,ID_CLINICA,DE_CLINICA,DT_DESFECHO,DE_DESFECHO,aa_nascimento,IC_SEXO,UTI,INT,PS
0,769E89D426E84A4797EF495608A0429E,261F2494FE76CC0F74C9A7A5A841C404,2020-11-26,Recepção do Centro Diagnóstico,17 Alfa Hidroxiprogesterona,17-Alfa-Hidroxiprog.,35,2020-11-26,Externo,20.0,Procedimentos,2020-11-26,Alta Administrativa,1974,F,0,0,1
1,9DECD77C85AD8EF58C52F5D747546632,F1C30774A4B98ECA55D2DF430DFA0025,2020-06-10,Recepção do Centro Diagnóstico,17 Hidroxipregnenolona,17-Hidroxi Pregnenolona,44,2020-06-10,Externo,20.0,Procedimentos,2020-06-10,Alta Administrativa,1982,F,0,0,1
2,CE22DCFCE0BF5718B1781E4F01643661,53355F35A7D8DBE10777D1C66A58D05E,2020-05-26,Laboratório de Patologia Clínica,"Acido Ascorbico, plasma","Ácido ascorbico, plasma","0,8",2020-05-26,Externo,20.0,Procedimentos,2020-05-26,Alta Administrativa,1984,F,0,0,1
3,9A76D584342057BB53DA0E2032E45DE5,82061D7F2BAFE205BF95643D2190A50A,2020-10-30,Ultrassonografia,"Acido Ascorbico, plasma","Ácido ascorbico, plasma","0,8",2020-10-30,Externo,20.0,Procedimentos,2020-10-30,Alta Administrativa,1993,M,0,0,1
4,946A2C9536DF3A202452FD18E5F35994,B3DA672B0CDFD47DD20ADD7ECB0CBD7C,2020-11-04,Recepção do Centro Diagnóstico,"Acido Ascorbico, plasma","Ácido ascorbico, plasma","1,0",2020-11-04,Externo,20.0,Procedimentos,2020-11-04,Alta Administrativa,1982,F,0,0,1


In [ ]:
# juntando os exemplos de UTI, INT e PS em um dataset só

df_grupos = pd.concat([exemplos_UTI, exemplos_INT, exemplos_PS])

df_grupos[df_grupos['UTI']==1].head(2)

,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,ID_CLINICA,DE_CLINICA,DT_DESFECHO,DE_DESFECHO,aa_nascimento,IC_SEXO,UTI,INT,PS
109,CE30E08E37887C8B936E24D36D9CDE85,ADE8CE496E52B77B70699907940F739E,2020-09-26,UTI,Acido Fólico,Ácido fólico,"3,1",2020-08-26,Internado,1.0,Clínica Médica,2020-09-26,Óbito nas primeiras 48hs de internação sem nec...,1943,M,1,0,0
110,70ECA50821E10D1B747212B96A9FADB9,91D503356F2073546A847A9F8ABCF674,2020-07-07,UTI,Acido Fólico,Ácido fólico,"9,9",2020-05-07,Internado,1.0,Clínica Médica,2020-07-07,Alta médica melhorado,1951,F,1,0,0


In [ ]:
df_grupos[df_grupos['INT']==1].head(2)

,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,ID_CLINICA,DE_CLINICA,DT_DESFECHO,DE_DESFECHO,aa_nascimento,IC_SEXO,UTI,INT,PS
75,369C7BA557BDD9BD5A50EA5059A6C012,6A53F7CF2700AC25AE276DB792A21E25,2020-12-16,Unidades de Internação,Acido Fólico,Ácido fólico,"4,5",2020-12-11,Internado,1.0,Clínica Médica,2020-12-16,Alta médica melhorado,1935,M,0,1,0
76,D4D3CF95BAD0955F0123D95E1ABC124C,A4C48142455EB43F3FD280056ED0B163,2020-11-11,Unidades de Internação,Acido Fólico,Ácido fólico,"7,1",2020-02-10,Internado,8.0,Ambulância,2020-11-11,Alta médica melhorado,1940,M,0,1,0


In [ ]:
#Passando para csv
#df_grupos.to_csv('dados_grupos.csv', sep='\t', encoding='utf-8')

#### **FILTRO 3: CRIANDO DATAFRAME COM EXAMES APENAS PROVINDOS DO PS**

In [ ]:
df_PS = df_grupos[df_grupos['PS']==1]

df_PS.head(2)

,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,ID_CLINICA,DE_CLINICA,DT_DESFECHO,DE_DESFECHO,aa_nascimento,IC_SEXO,UTI,INT,PS
0,769E89D426E84A4797EF495608A0429E,261F2494FE76CC0F74C9A7A5A841C404,2020-11-26,Recepção do Centro Diagnóstico,17 Alfa Hidroxiprogesterona,17-Alfa-Hidroxiprog.,35,2020-11-26,Externo,20.0,Procedimentos,2020-11-26,Alta Administrativa,1974,F,0,0,1
1,9DECD77C85AD8EF58C52F5D747546632,F1C30774A4B98ECA55D2DF430DFA0025,2020-06-10,Recepção do Centro Diagnóstico,17 Hidroxipregnenolona,17-Hidroxi Pregnenolona,44,2020-06-10,Externo,20.0,Procedimentos,2020-06-10,Alta Administrativa,1982,F,0,0,1


In [ ]:
df_PS.shape

(372742, 18)

In [ ]:
df_PS['ID_PACIENTE'].nunique()

8222

In [ ]:
#df_PS.to_csv('ps.csv', sep='\t', encoding='utf-8')

#### **PACIENTES vs FLAGS (INT, PS, UTI)**

In [ ]:
#Criando um agrupamento em função do ID_Paciente e agregando valores das flags 'UTI', 'INT' e 'PS'

df_hsl_1 = df_grupos.pivot_table(index='ID_PACIENTE', values=['UTI','INT','PS'], columns=[], aggfunc='sum')

print(df_hsl_1.shape)

df_hsl_1.head(10)


(8750, 3)


,INT,PS,UTI
ID_PACIENTE,,,
000150DB429AFF026ECD130B0A076CEF,0,2,0
00017961865C4F766FDBB3CD8FE0BFB0,0,45,0
000F0BC139D2846DB86AA32B8F05B215,0,61,0
00177290279939FB33386B29198C450E,0,1,0
001BEECDD4D16B6F3ECDBE57F48E7982,0,2,0
0028785949D91BD93442838FC898E229,0,10,0
002B919CC409B11DE52FB212379BE2CB,0,133,0
003051C9B19101D1C10C5DC654384017,0,79,0
003F1F4C194763E4A00FD809AF5FA6AA,176,90,0


In [ ]:
#gera csv
#df_hsl_1.to_csv('pacientes_flag.csv', sep='\t', encoding='utf-8')

In [ ]:
# transformando o index em uma coluna

df_hsl_1 = df_hsl_1.reset_index()

df_hsl_1.head()

,ID_PACIENTE,INT,PS,UTI
0,000150DB429AFF026ECD130B0A076CEF,0,2,0
1,00017961865C4F766FDBB3CD8FE0BFB0,0,45,0
2,000F0BC139D2846DB86AA32B8F05B215,0,61,0
3,00177290279939FB33386B29198C450E,0,1,0
4,001BEECDD4D16B6F3ECDBE57F48E7982,0,2,0



#### **PACIENTES GRUPO_0**

In [ ]:
# Selecionando pacientes do grupo 0

grupo_0 = df_hsl_1.query('PS > 0 & INT == 0 & UTI == 0')

print(grupo_0.head(3), '\n')

pacientes_grupo_0 = grupo_0['ID_PACIENTE'].unique()

print(pacientes_grupo_0,'\n')

print('quantidade de pacientes no GRUPO_0: ', len(pacientes_grupo_0))


                        ID_PACIENTE  INT  PS  UTI
0  000150DB429AFF026ECD130B0A076CEF    0   2    0
1  00017961865C4F766FDBB3CD8FE0BFB0    0  45    0
2  000F0BC139D2846DB86AA32B8F05B215    0  61    0 

['000150DB429AFF026ECD130B0A076CEF' '00017961865C4F766FDBB3CD8FE0BFB0'
 '000F0BC139D2846DB86AA32B8F05B215' ... 'FFEE2A236442ECF6D9042F39A8EA73FD'
 'FFF5753408C98D5E0218931420B6AF85' 'FFFA1AB079C74F2EFEA6BA07E5B5DFC9'] 

quantidade de pacientes no GRUPO_0:  7092


In [ ]:
#Criando o Grupo_0 a partir do dataframe df_PS

GRUPO_0 = df_PS.loc[df_hsl['ID_PACIENTE'].isin(pacientes_grupo_0)]

GRUPO_0['GRUPO'] = 'GRUPO_0'

GRUPO_0.head(2)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,ID_CLINICA,DE_CLINICA,DT_DESFECHO,DE_DESFECHO,aa_nascimento,IC_SEXO,UTI,INT,PS,GRUPO
1,9DECD77C85AD8EF58C52F5D747546632,F1C30774A4B98ECA55D2DF430DFA0025,2020-06-10,Recepção do Centro Diagnóstico,17 Hidroxipregnenolona,17-Hidroxi Pregnenolona,44,2020-06-10,Externo,20.0,Procedimentos,2020-06-10,Alta Administrativa,1982,F,0,0,1,GRUPO_0
2,CE22DCFCE0BF5718B1781E4F01643661,53355F35A7D8DBE10777D1C66A58D05E,2020-05-26,Laboratório de Patologia Clínica,"Acido Ascorbico, plasma","Ácido ascorbico, plasma","0,8",2020-05-26,Externo,20.0,Procedimentos,2020-05-26,Alta Administrativa,1984,F,0,0,1,GRUPO_0


#### **PACIENTES GRUPO_1**

In [ ]:
# Selecionando pacientes do grupo 1

grupo_1 = df_hsl_1.query('PS > 0 & INT > 0 & UTI == 0')

print(grupo_1.head(3), '\n')

pacientes_grupo_1 = grupo_1['ID_PACIENTE'].unique()

#print(pacientes_grupo_1,'\n')

print('quantidade de pacientes no GRUPO_1: ', len(pacientes_grupo_1))

                         ID_PACIENTE  INT   PS  UTI
8   003F1F4C194763E4A00FD809AF5FA6AA  176   90    0
47  0162D896109ED193BDE84532A63C4CFF  223  236    0
57  01A1C39530DE1A97D9F368049FE20AA3   81  111    0 

quantidade de pacientes no GRUPO_1:  618


In [ ]:
#Criando o Grupo_1 a partir do dataframe df_PS

GRUPO_1 = df_PS.loc[df_hsl['ID_PACIENTE'].isin(pacientes_grupo_1)]

GRUPO_1['GRUPO'] = 'GRUPO_1'

GRUPO_1.head(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,ID_CLINICA,DE_CLINICA,DT_DESFECHO,DE_DESFECHO,aa_nascimento,IC_SEXO,UTI,INT,PS,GRUPO
17,F7485478A910C42521C79D6400FC3BA0,229D7D60D205EF5A0E0BBA5E7083AB73,2020-09-22,Laboratório de Patologia Clínica,Acido Fólico,Ácido fólico,"11,4",2020-09-22,Externo,31.0,Exames,2020-09-22,Alta Administrativa,1939,M,0,0,1,GRUPO_1
19,E8766C1FD5F8FDAF26EB9B254CF4F52B,7E44EDC86179A49033F3A7BBFF20BB8F,2020-01-09,Laboratório de Patologia Clínica,Acido Fólico,Ácido fólico,"11,9",2020-01-09,Externo,20.0,Procedimentos,2020-01-09,Alta Administrativa,1992,M,0,0,1,GRUPO_1
21,72FA83AC298CCEF9CA86D1080D21A313,9E44FDC9022850D9813428D479356A6E,2020-05-19,Laboratório de Patologia Clínica,Acido Fólico,Ácido fólico,"10,4",2020-05-19,Externo,20.0,Procedimentos,2020-05-19,Alta Administrativa,1945,M,0,0,1,GRUPO_1


#### **PACIENTES GRUPO_2**

In [ ]:
# Selecionando pacientes grupo 2

grupo_2 = df_hsl_1.query('INT == 0 & PS > 0 & UTI > 0')

print(grupo_2.head(5), '\n')

pacientes_grupo_2 = grupo_2['ID_PACIENTE'].unique()

#print(pacientes_grupo_2,'\n')

print('Quantidade de pacientes no GRUPO_2: ', len(pacientes_grupo_2))

                          ID_PACIENTE  INT   PS  UTI
11   0047AF5116BC8AC8EFE6BBB98DA14DFA    0   63  245
34   00F5BA3CAE9B4462033138372DBC5267    0    1  311
137  03EFA96A0168B0FF5CB433E0710491A5    0    1  250
298  088557EE5C8A7225C7FBF3AC5CAC4316    0    8  213
417  0BF770E4CF7DA296BA6FEAF769746EB8    0  147   88 

Quantidade de pacientes no GRUPO_2:  112


In [ ]:
#Criando o Grupo_2 a partir do dataframe df_PS

GRUPO_2 = df_PS.loc[df_hsl['ID_PACIENTE'].isin(pacientes_grupo_2)]

GRUPO_2['GRUPO'] = 'GRUPO_2'

print(GRUPO_2['ID_PACIENTE'].nunique(), '\n')

GRUPO_2.head(3)

112 



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,ID_CLINICA,DE_CLINICA,DT_DESFECHO,DE_DESFECHO,aa_nascimento,IC_SEXO,UTI,INT,PS,GRUPO
643,3E6D7A10CA4F0974E403455556ACD5C8,D25FF511EFD840801E34344B160DEC45,2020-06-19,Holter,Bilirrubinas Total E Fracoes,Bilirrubina Indireta,"0,46",2020-06-19,Externo,20.0,Procedimentos,2020-06-19,Alta Administrativa,1951,M,0,0,1,GRUPO_2
724,56CA7ADDE571980E45E39E05D9F5E2B5,1B69DD9DD4F1AF513634BB70E1037A0C,2020-05-12,Pronto Socorro,Bilirrubinas Total E Fracoes,Bilirrubina Indireta,"0,13",2020-05-12,Pronto Atendimento,42.0,CL Médica Síndromes Virais,2020-05-12,Alta médica melhorado,1947,M,0,0,1,GRUPO_2
737,4C28DB6CC0CA101CAA574083CF238466,49E4135B80C78D2F0DB6CC9355EEB65D,2020-08-18,Pronto Socorro,Bilirrubinas Total E Fracoes,Bilirrubina Indireta,"0,14",2020-08-18,Pronto Atendimento,42.0,CL Médica Síndromes Virais,2020-08-18,Alta médica melhorado,1961,M,0,0,1,GRUPO_2


#### **PACIENTES GRUPO_3**

In [ ]:
# Selecionando pacientes grupo 3

grupo_3 = df_hsl_1.query('UTI > 0 & PS > 0 & INT > 0')

print(grupo_3.head(5), '\n')

pacientes_grupo_3 = grupo_3['ID_PACIENTE'].unique()

#print(pacientes_grupo_3,'\n')

print('Quantidade de pacientes no GRUPO_3: ', len(pacientes_grupo_3))

                         ID_PACIENTE   INT   PS   UTI
10  004688799FD293C3ABE0A07209FD8B75   267  330   899
19  009F0D6B3BA6C0E2D406585697D679EB   117   89   308
39  010AF0C3418C765CDA28B5957210A819   179   77   319
85  028D6E2C0A51B4E2D3B919DB94D6BCB2  1789   62  1197
98  02F2F8FC72B4714F5C6B98C04EDB5913   426   85    59 

Quantidade de pacientes no GRUPO_3:  400


In [ ]:
#Criando o Grupo_3 a partir do dataframe df_PS

GRUPO_3 = df_PS.loc[df_hsl['ID_PACIENTE'].isin(pacientes_grupo_3)]

GRUPO_3['GRUPO'] = 'GRUPO_3'

print(GRUPO_3['ID_PACIENTE'].nunique(), '\n')

GRUPO_3.head(3)

400 



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,ID_CLINICA,DE_CLINICA,DT_DESFECHO,DE_DESFECHO,aa_nascimento,IC_SEXO,UTI,INT,PS,GRUPO
0,769E89D426E84A4797EF495608A0429E,261F2494FE76CC0F74C9A7A5A841C404,2020-11-26,Recepção do Centro Diagnóstico,17 Alfa Hidroxiprogesterona,17-Alfa-Hidroxiprog.,35,2020-11-26,Externo,20.0,Procedimentos,2020-11-26,Alta Administrativa,1974,F,0,0,1,GRUPO_3
11,0375A54F4BAD09CEA28A86FEB1FD37AD,40696A8D51865A2155BD563C2D4F3226,2020-11-06,Recepção do Centro Diagnóstico,"Acido Ascorbico, plasma","Ácido ascorbico, plasma","1,0",2020-11-06,Externo,20.0,Procedimentos,2020-11-06,Alta Administrativa,1950,M,0,0,1,GRUPO_3
12,369C7BA557BDD9BD5A50EA5059A6C012,608E5E827F75BB92250DCA5DAD70C383,2020-08-25,Hemodiálise,Acido Fólico,Ácido fólico,"10,2",2020-08-24,Ambulatorial,15.0,Hemodiálise,2020-08-25,Alta Administrativa,1935,M,0,0,1,GRUPO_3


#### **OUTROS PACIENTES**

OBS: ESSES PACIENTES SERÃO DESCONSIDERADOS, POIS NÃO POSSUIEM EXAMES QUE INDIQUEM PREDISPOSIÇÃO INCIAL

In [ ]:
# Verificando os pacientes que não possuem exames em PS, esses pacientes foram desconsiderados

grupo_4 = df_hsl_1.query('PS == 0')

print(grupo_4.head(5), '\n')

pacientes_grupo_4 = grupo_4['ID_PACIENTE'].unique()

#print(pacientes_grupo_2,'\n')

print('Quantidade de pacientes no GRUPO_4: ', len(pacientes_grupo_4))

                          ID_PACIENTE  INT  PS   UTI
43   01451931334246A7DE4F71DEE7710859  113   0     0
53   0183BA4D9368936BAD131398B55CDDC3  603   0  1495
58   01A29BBFDC18988C5200E74AE169841E  155   0     0
86   0296EEF7845CE2C5DAB358E894256092    1   0   730
121  03921BD7EFD5787934B8900682F73608    0   0   437 

Quantidade de pacientes no GRUPO_4:  528


#### **JUNTANDO OS GRUPOS EM UM SÓ DATASET**

In [ ]:
df_sirio_libanes = pd.concat([GRUPO_0, GRUPO_1, GRUPO_2, GRUPO_3])

df_sirio_libanes.head(2)

,ID_PACIENTE,ID_ATENDIMENTO,DT_COLETA,DE_ORIGEM,DE_EXAME,DE_ANALITO,DE_RESULTADO,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,ID_CLINICA,DE_CLINICA,DT_DESFECHO,DE_DESFECHO,aa_nascimento,IC_SEXO,UTI,INT,PS,GRUPO
1,9DECD77C85AD8EF58C52F5D747546632,F1C30774A4B98ECA55D2DF430DFA0025,2020-06-10,Recepção do Centro Diagnóstico,17 Hidroxipregnenolona,17-Hidroxi Pregnenolona,44,2020-06-10,Externo,20.0,Procedimentos,2020-06-10,Alta Administrativa,1982,F,0,0,1,GRUPO_0
2,CE22DCFCE0BF5718B1781E4F01643661,53355F35A7D8DBE10777D1C66A58D05E,2020-05-26,Laboratório de Patologia Clínica,"Acido Ascorbico, plasma","Ácido ascorbico, plasma","0,8",2020-05-26,Externo,20.0,Procedimentos,2020-05-26,Alta Administrativa,1984,F,0,0,1,GRUPO_0


In [ ]:
df_sirio_libanes.shape

(372742, 19)

In [ ]:
df_sirio_libanes['ID_PACIENTE'].nunique()

8222

In [ ]:
G0 = df_sirio_libanes[df_sirio_libanes['GRUPO']=='GRUPO_0']
print('GRUPO_0: ', G0['ID_PACIENTE'].nunique())

G1 = df_sirio_libanes[df_sirio_libanes['GRUPO']=='GRUPO_1']
print('GRUPO_1: ', G1['ID_PACIENTE'].nunique())

G2 = df_sirio_libanes[df_sirio_libanes['GRUPO']=='GRUPO_2']
print('GRUPO_2: ', G2['ID_PACIENTE'].nunique())

G3 = df_sirio_libanes[df_sirio_libanes['GRUPO']=='GRUPO_3']
print('GRUPO_3: ', G3['ID_PACIENTE'].nunique())

GRUPO_0:  7092
GRUPO_1:  618
GRUPO_2:  112
GRUPO_3:  400


#### **FILTRO 4: NOVA SELEÇÃO MANUAL DE ATRIBUTOS**

In [ ]:
#deletando colunas que não serão mais necesárias

df_sirio = df_sirio_libanes.drop(columns=['ID_ATENDIMENTO','DE_ORIGEM','ID_CLINICA','DE_CLINICA', 'DT_DESFECHO', 'UTI','INT','PS'])

df_sirio.head(3)

,ID_PACIENTE,DT_COLETA,DE_EXAME,DE_ANALITO,DE_RESULTADO,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,DE_DESFECHO,aa_nascimento,IC_SEXO,GRUPO
1,9DECD77C85AD8EF58C52F5D747546632,2020-06-10,17 Hidroxipregnenolona,17-Hidroxi Pregnenolona,44,2020-06-10,Externo,Alta Administrativa,1982,F,GRUPO_0
2,CE22DCFCE0BF5718B1781E4F01643661,2020-05-26,"Acido Ascorbico, plasma","Ácido ascorbico, plasma","0,8",2020-05-26,Externo,Alta Administrativa,1984,F,GRUPO_0
3,9A76D584342057BB53DA0E2032E45DE5,2020-10-30,"Acido Ascorbico, plasma","Ácido ascorbico, plasma","0,8",2020-10-30,Externo,Alta Administrativa,1993,M,GRUPO_0


In [ ]:
df_sirio.shape

(372742, 11)

#### **FILTRO 5: SELECIONANDO EXAMES COM ATÉ 3 DIAS APÓS O ATENDIMENTO** 

(OU SEJA, DATA DE COLETA - DATA DE ATENDIMENTO <= 3 DIAS)

In [ ]:
#criando coluna com período de exames
df_sirio['PERIODO_EXAMES'] = (df_sirio['DT_COLETA']-df_sirio['DT_ATENDIMENTO']).dt.days
print(df_sirio.shape)
df_sirio.head(3)

(372742, 12)


,ID_PACIENTE,DT_COLETA,DE_EXAME,DE_ANALITO,DE_RESULTADO,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,DE_DESFECHO,aa_nascimento,IC_SEXO,GRUPO,PERIODO_EXAMES
1,9DECD77C85AD8EF58C52F5D747546632,2020-06-10,17 Hidroxipregnenolona,17-Hidroxi Pregnenolona,44,2020-06-10,Externo,Alta Administrativa,1982,F,GRUPO_0,0.0
2,CE22DCFCE0BF5718B1781E4F01643661,2020-05-26,"Acido Ascorbico, plasma","Ácido ascorbico, plasma","0,8",2020-05-26,Externo,Alta Administrativa,1984,F,GRUPO_0,0.0
3,9A76D584342057BB53DA0E2032E45DE5,2020-10-30,"Acido Ascorbico, plasma","Ácido ascorbico, plasma","0,8",2020-10-30,Externo,Alta Administrativa,1993,M,GRUPO_0,0.0


In [ ]:
#Selecionando apenas exemplos de exames com até três dias da entrada no hospital

tres_dias = df_sirio[df_sirio['PERIODO_EXAMES']<=3]

print(tres_dias.shape)

tres_dias['PERIODO_EXAMES'].value_counts()

(363079, 12)


 0.0      349970
 1.0        6889
 2.0        1552
 3.0         714
-86.0        425
           ...  
-283.0         3
-17.0          2
-51.0          2
-22.0          1
-20.0          1
Name: PERIODO_EXAMES, Length: 73, dtype: int64

In [ ]:
#investigando 'PERIODO_EXAMES' negativos

# Os intervalos negativos significam que DT_COLETA é anterior a DT_ATENDIMENTO.

x = tres_dias[tres_dias['PERIODO_EXAMES']<0]

x.head(3)

,ID_PACIENTE,DT_COLETA,DE_EXAME,DE_ANALITO,DE_RESULTADO,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,DE_DESFECHO,aa_nascimento,IC_SEXO,GRUPO,PERIODO_EXAMES
1494,A812B082EE43AFA716B6F9C33145F8EE,2020-05-13,Bilirrubinas Total E Fracoes,Bilirrubina Indireta,"0,02",2020-12-05,Ambulatorial,Alta Administrativa,1966,F,GRUPO_0,-206.0
2540,A812B082EE43AFA716B6F9C33145F8EE,2020-05-13,Bilirrubinas Total E Fracoes,Bilirrubina Total,"0,07",2020-12-05,Ambulatorial,Alta Administrativa,1966,F,GRUPO_0,-206.0
3584,A812B082EE43AFA716B6F9C33145F8EE,2020-05-13,Bilirrubinas Total E Fracoes,Bilirrubina Direta,"0,05",2020-12-05,Ambulatorial,Alta Administrativa,1966,F,GRUPO_0,-206.0


In [ ]:
#pegando um paciente como exemplo

x[x['ID_PACIENTE']=='A812B082EE43AFA716B6F9C33145F8EE']

,ID_PACIENTE,DT_COLETA,DE_EXAME,DE_ANALITO,DE_RESULTADO,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,DE_DESFECHO,aa_nascimento,IC_SEXO,GRUPO,PERIODO_EXAMES
1494,A812B082EE43AFA716B6F9C33145F8EE,2020-05-13,Bilirrubinas Total E Fracoes,Bilirrubina Indireta,"0,02",2020-12-05,Ambulatorial,Alta Administrativa,1966,F,GRUPO_0,-206.0
2540,A812B082EE43AFA716B6F9C33145F8EE,2020-05-13,Bilirrubinas Total E Fracoes,Bilirrubina Total,"0,07",2020-12-05,Ambulatorial,Alta Administrativa,1966,F,GRUPO_0,-206.0
3584,A812B082EE43AFA716B6F9C33145F8EE,2020-05-13,Bilirrubinas Total E Fracoes,Bilirrubina Direta,"0,05",2020-12-05,Ambulatorial,Alta Administrativa,1966,F,GRUPO_0,-206.0
5685,A812B082EE43AFA716B6F9C33145F8EE,2020-05-13,Tgo,AST (TGO),19,2020-12-05,Ambulatorial,Alta Administrativa,1966,F,GRUPO_0,-206.0
7159,A812B082EE43AFA716B6F9C33145F8EE,2020-05-13,Tgp,ALT (TGP),12,2020-12-05,Ambulatorial,Alta Administrativa,1966,F,GRUPO_0,-206.0
9632,A812B082EE43AFA716B6F9C33145F8EE,2020-05-13,Fosfatase Alcalina,Fosfatase Alcalina,113,2020-12-05,Ambulatorial,Alta Administrativa,1966,F,GRUPO_0,-206.0
13298,A812B082EE43AFA716B6F9C33145F8EE,2020-05-13,Uréia,Uréia,18,2020-12-05,Ambulatorial,Alta Administrativa,1966,F,GRUPO_0,-206.0
18034,A812B082EE43AFA716B6F9C33145F8EE,2020-05-13,"Proteína C Reativa, plasma",Proteína C-Reativa,"5,43",2020-12-05,Ambulatorial,Alta Administrativa,1966,F,GRUPO_0,-206.0
24463,A812B082EE43AFA716B6F9C33145F8EE,2020-05-13,Cálcio Ionizável,Calcio Ionizavel,"1,29",2020-12-05,Ambulatorial,Alta Administrativa,1966,F,GRUPO_0,-206.0
28029,A812B082EE43AFA716B6F9C33145F8EE,2020-05-13,Fósforo,Fósforo,"2,6",2020-12-05,Ambulatorial,Alta Administrativa,1966,F,GRUPO_0,-206.0


Os pacientes com período_exames negativo possuem data de coleta anterior ao atendimento, o que pode ser uma inconsistência.

In [ ]:
# Verificando quais são os desfechos desses exemplos:
x['DE_DESFECHO'].value_counts()

Alta Administrativa                                                 3412
Alta médica melhorado                                                436
Óbito após 48hs de internação sem necrópsia                           96
Óbito nas primeiras 48hs de internação sem necrópsia não agônico      10
Name: DE_DESFECHO, dtype: int64

In [ ]:
#eliminando exemplos com 'PERIODO_EXAMES' negativos

tres_dias = tres_dias[tres_dias['PERIODO_EXAMES']>=0]

print(tres_dias['PERIODO_EXAMES'].value_counts())

print('\n', tres_dias.shape)

0.0    349970
1.0      6889
2.0      1552
3.0       714
Name: PERIODO_EXAMES, dtype: int64

 (359125, 12)


In [ ]:
tres_dias['DE_DESFECHO'].value_counts()

Alta Administrativa                                                 217892
Alta médica melhorado                                               134256
Alta a pedido                                                         3241
Alta médica Inalterado                                                1646
Alta por abandono                                                      623
Óbito após 48hs de internação sem necrópsia                            479
Alta médica curado                                                     400
Transferência Inter-Hospitalar Externa - Serviço de Ambulância         286
Óbito nas primeiras 48hs de internação sem necrópsia não agônico       157
Transferência Inter-Hospitalar Externa - Transporte Próprio             87
Óbito nas primeiras 48hs de internação sem necrópsia agônico            55
Desistência do atendimento                                               3
Name: DE_DESFECHO, dtype: int64

In [ ]:
#investigando quantos pacientes unicos em cada tipo de desfecho

pacientes = tres_dias['ID_PACIENTE'].unique()

print('Pacientes unicos com exames de 0 a 3 dias: ', pacientes.shape, '\n')

pacientes = tres_dias.drop_duplicates(subset='ID_PACIENTE', keep='first')

pacientes['DE_DESFECHO'].value_counts()


Pacientes unicos com exames de 0 a 3 dias:  (8185,) 



Alta Administrativa                                                 4650
Alta médica melhorado                                               3369
Alta a pedido                                                         73
Alta médica Inalterado                                                63
Alta médica curado                                                     9
Alta por abandono                                                      7
Óbito após 48hs de internação sem necrópsia                            6
Transferência Inter-Hospitalar Externa - Serviço de Ambulância         3
Óbito nas primeiras 48hs de internação sem necrópsia não agônico       2
Desistência do atendimento                                             1
Transferência Inter-Hospitalar Externa - Transporte Próprio            1
Óbito nas primeiras 48hs de internação sem necrópsia agônico           1
Name: DE_DESFECHO, dtype: int64

In [ ]:
df_hsl = tres_dias

df_hsl.head(2)

,ID_PACIENTE,DT_COLETA,DE_EXAME,DE_ANALITO,DE_RESULTADO,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,DE_DESFECHO,aa_nascimento,IC_SEXO,GRUPO,PERIODO_EXAMES
1,9DECD77C85AD8EF58C52F5D747546632,2020-06-10,17 Hidroxipregnenolona,17-Hidroxi Pregnenolona,44,2020-06-10,Externo,Alta Administrativa,1982,F,GRUPO_0,0.0
2,CE22DCFCE0BF5718B1781E4F01643661,2020-05-26,"Acido Ascorbico, plasma","Ácido ascorbico, plasma","0,8",2020-05-26,Externo,Alta Administrativa,1984,F,GRUPO_0,0.0


In [ ]:
df_hsl.shape

(359125, 12)

In [ ]:
df_hsl['ID_PACIENTE'].nunique()

8185

#### **ÓBITOS**

In [ ]:
#Verificando se todos os óbitos estão no grupo grave

obitos = ['Óbito após 48hs de internação sem necrópsia',
          'Óbito nas primeiras 48hs de internação sem necrópsia não agônico',
          'Óbito nas primeiras 48hs de internação sem necrópsia agônico']

df_obitos = df_hsl.loc[df_hsl['DE_DESFECHO'].isin(obitos)]

df_obitos['GRUPO'].value_counts()

GRUPO_3    515
GRUPO_2    176
Name: GRUPO, dtype: int64

In [ ]:
obitos_GRUPO_1 = df_obitos[df_obitos['GRUPO']=='GRUPO_1']

obitos_GRUPO_1['ID_PACIENTE'].nunique()

#esses pacientes precisão passar para o grupo 2, pois dem ter gravidade máxima

obitos_GRUPO_1['ID_PACIENTE'].unique()

array([], dtype=object)

In [ ]:
df_hsl.head(50)

,ID_PACIENTE,DT_COLETA,DE_EXAME,DE_ANALITO,DE_RESULTADO,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,DE_DESFECHO,aa_nascimento,IC_SEXO,GRUPO,PERIODO_EXAMES
1,9DECD77C85AD8EF58C52F5D747546632,2020-06-10,17 Hidroxipregnenolona,17-Hidroxi Pregnenolona,44,2020-06-10,Externo,Alta Administrativa,1982,F,GRUPO_0,0.0
2,CE22DCFCE0BF5718B1781E4F01643661,2020-05-26,"Acido Ascorbico, plasma","Ácido ascorbico, plasma","0,8",2020-05-26,Externo,Alta Administrativa,1984,F,GRUPO_0,0.0
3,9A76D584342057BB53DA0E2032E45DE5,2020-10-30,"Acido Ascorbico, plasma","Ácido ascorbico, plasma","0,8",2020-10-30,Externo,Alta Administrativa,1993,M,GRUPO_0,0.0
4,946A2C9536DF3A202452FD18E5F35994,2020-11-04,"Acido Ascorbico, plasma","Ácido ascorbico, plasma","1,0",2020-11-04,Externo,Alta Administrativa,1982,F,GRUPO_0,0.0
5,D2F8B9BBF29FD23AC2E806D1EA3A99FC,2020-08-15,"Acido Ascorbico, plasma","Ácido ascorbico, plasma","1,7",2020-08-15,Externo,Alta Administrativa,1989,F,GRUPO_0,0.0
6,C86EB9BAFEEDB686D59461EFBD3AFE26,2020-05-22,"Acido Ascorbico, plasma","Ácido ascorbico, plasma","1,6",2020-05-22,Externo,Alta Administrativa,1972,F,GRUPO_0,0.0
7,1A2E6F3CBEBC107D186ECA82A8D3273F,2020-11-22,"Acido Ascorbico, plasma","Ácido ascorbico, plasma","1,9",2020-11-22,Externo,Alta Administrativa,1977,F,GRUPO_0,0.0
8,50EF49691C86C9F8AF416F8331EEFAF7,2020-02-10,"Acido Ascorbico, plasma","Ácido ascorbico, plasma","1,4",2020-02-10,Externo,Alta Administrativa,1973,M,GRUPO_0,0.0
9,05D2CF001DA4F6A91A5152BBE090A7C3,2020-05-16,"Acido Ascorbico, plasma","Ácido ascorbico, plasma","1,4",2020-05-16,Externo,Alta Administrativa,1983,F,GRUPO_0,0.0
10,420FDC7BF5E9C3CA3725F1EB6B1DFC9D,2020-09-16,"Acido Ascorbico, plasma","Ácido ascorbico, plasma","0,8",2020-09-16,Externo,Alta Administrativa,1962,F,GRUPO_0,0.0


#### **FILTRO 6: ELIMINANDO EXAMES COM RESULTADOS EM FORMA DE TEXTO**

https://www.vooo.pro/insights/12-tecnicas-pandas-uteis-em-python-para-manipulacao-de-dados/


In [ ]:
# Trocando ',' por '.'.

df_hsl['DE_RESULTADO'] = [x.replace(',', '.') for x in df_hsl['DE_RESULTADO']]  

resultados = df_hsl['DE_RESULTADO'].value_counts()

resultados.to_csv('resultados.csv', sep='\t',encoding='utf-8')

In [ ]:
#  Convertendo a coluna DE_RESULTADO para numérico
#  Deu erro, pois existem resultados não numericos
#  Então nos próximos passos serão excluídos os resultados não numéricos

#df_hsl['DE_RESULTADO'] = df_hsl['DE_RESULTADO'].astype(float)                 
#df_final.info()

In [ ]:
df_hsl.dtypes

ID_PACIENTE                    object
DT_COLETA              datetime64[ns]
DE_EXAME                       object
DE_ANALITO                     object
DE_RESULTADO                   object
DT_ATENDIMENTO         datetime64[ns]
DE_TIPO_ATENDIMENTO            object
DE_DESFECHO                    object
aa_nascimento                  object
IC_SEXO                        object
GRUPO                          object
PERIODO_EXAMES                float64
dtype: object

In [ ]:
#função para verificar se um variável é numérica

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [ ]:
num = '9999.999'

is_number(num)

True

In [ ]:
#Criando uma nova coluna para testar se o resultado é numérico

df_hsl['Tipo_resultado'] = ''

df_hsl.head(3)

,ID_PACIENTE,DT_COLETA,DE_EXAME,DE_ANALITO,DE_RESULTADO,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,DE_DESFECHO,aa_nascimento,IC_SEXO,GRUPO,PERIODO_EXAMES,Tipo_resultado
1,9DECD77C85AD8EF58C52F5D747546632,2020-06-10,17 Hidroxipregnenolona,17-Hidroxi Pregnenolona,44,2020-06-10,Externo,Alta Administrativa,1982,F,GRUPO_0,0.0,
2,CE22DCFCE0BF5718B1781E4F01643661,2020-05-26,"Acido Ascorbico, plasma","Ácido ascorbico, plasma",0.8,2020-05-26,Externo,Alta Administrativa,1984,F,GRUPO_0,0.0,
3,9A76D584342057BB53DA0E2032E45DE5,2020-10-30,"Acido Ascorbico, plasma","Ácido ascorbico, plasma",0.8,2020-10-30,Externo,Alta Administrativa,1993,M,GRUPO_0,0.0,


O código abaixo verifica linha por linha se a variável é numérica, ou seja, se poderá ser convertida em float. 

In [ ]:
# Este bloco funciona, porém é demorado.
"""
for index, row in df_hsl.iterrows():
        verifica = is_number(row['DE_RESULTADO'])
        if verifica == True:
            df_hsl.loc[index,'Tipo_resultado'] =  True
    
        else:
            df_hsl.loc[index,'Tipo_resultado'] = False

# gera arquivo: df_hsl_true_false.csv
df_hsl.to_csv('df_hsl_true_false.csv', sep=';',encoding='utf-8')
"""

"\nfor index, row in df_hsl.iterrows():\n        verifica = is_number(row['DE_RESULTADO'])\n        if verifica == True:\n            df_hsl.loc[index,'Tipo_resultado'] =  True\n    \n        else:\n            df_hsl.loc[index,'Tipo_resultado'] = False\n\ndf_hsl.to_csv('df_hsl_true_false.csv', sep=';',encoding='utf-8')\n"

In [ ]:
# lê arquivo: df_hsl_true_false.csv
df_hsl_1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/2021 dezembro Artigo/Arquivos_v3/df_hsl_true_false_v3.csv', sep=';')

In [ ]:
df_hsl_1.head(3)

,Unnamed: 0,ID_PACIENTE,DT_COLETA,DE_EXAME,DE_ANALITO,DE_RESULTADO,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,DE_DESFECHO,aa_nascimento,IC_SEXO,GRUPO,PERIODO_EXAMES,Tipo_resultado
0,1,9DECD77C85AD8EF58C52F5D747546632,2020-06-10,17 Hidroxipregnenolona,17-Hidroxi Pregnenolona,44,2020-06-10,Externo,Alta Administrativa,1982,F,GRUPO_0,0.0,True
1,2,CE22DCFCE0BF5718B1781E4F01643661,2020-05-26,"Acido Ascorbico, plasma","Ácido ascorbico, plasma",0.8,2020-05-26,Externo,Alta Administrativa,1984,F,GRUPO_0,0.0,True
2,3,9A76D584342057BB53DA0E2032E45DE5,2020-10-30,"Acido Ascorbico, plasma","Ácido ascorbico, plasma",0.8,2020-10-30,Externo,Alta Administrativa,1993,M,GRUPO_0,0.0,True


In [ ]:
df_hsl_1 = df_hsl_1.drop(columns=['Unnamed: 0'])
df_hsl_1.head(3)

,ID_PACIENTE,DT_COLETA,DE_EXAME,DE_ANALITO,DE_RESULTADO,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,DE_DESFECHO,aa_nascimento,IC_SEXO,GRUPO,PERIODO_EXAMES,Tipo_resultado
0,9DECD77C85AD8EF58C52F5D747546632,2020-06-10,17 Hidroxipregnenolona,17-Hidroxi Pregnenolona,44,2020-06-10,Externo,Alta Administrativa,1982,F,GRUPO_0,0.0,True
1,CE22DCFCE0BF5718B1781E4F01643661,2020-05-26,"Acido Ascorbico, plasma","Ácido ascorbico, plasma",0.8,2020-05-26,Externo,Alta Administrativa,1984,F,GRUPO_0,0.0,True
2,9A76D584342057BB53DA0E2032E45DE5,2020-10-30,"Acido Ascorbico, plasma","Ácido ascorbico, plasma",0.8,2020-10-30,Externo,Alta Administrativa,1993,M,GRUPO_0,0.0,True


In [ ]:
df_hsl_1.shape

(359125, 13)

In [ ]:
df_hsl_1[df_hsl_1['Tipo_resultado']==False]

,ID_PACIENTE,DT_COLETA,DE_EXAME,DE_ANALITO,DE_RESULTADO,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,DE_DESFECHO,aa_nascimento,IC_SEXO,GRUPO,PERIODO_EXAMES,Tipo_resultado
10,E11A6F20607898576FE0EA3CB446DF5D,2020-08-09,Acido Fólico,Ácido fólico,superior a 24,2020-08-09,Externo,Alta Administrativa,1985,M,GRUPO_0,0.0,False
24,946A2C9536DF3A202452FD18E5F35994,2020-11-04,Acido Fólico,Ácido fólico,superior a 24,2020-11-04,Externo,Alta Administrativa,1982,F,GRUPO_0,0.0,False
36,49BB8D44889B35FC856E9121B21DF839,2020-05-03,Acido Fólico,Ácido fólico,superior a 24,2020-05-03,Externo,Alta Administrativa,1971,F,GRUPO_0,0.0,False
64,5375632434842C97BD3BD355C4CBFD91,2020-05-20,"Adenovirus, Sorologia",Adenovírus,inferior a 1/8,2020-05-20,Pronto Atendimento,Alta médica melhorado,1959,M,GRUPO_0,0.0,False
65,E8AE0A011B3D86AD76EDA62CDCED9F9D,2020-04-08,Anticoagulante Lúpico,Anticoagulante Lupico - Conclusão,ausência de anticoagulante lúpico.,2020-04-08,Externo,Alta Administrativa,1981,F,GRUPO_0,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
359120,4AF863B9C598125B9E27296C851AEFCB,2020-01-06,Urina Tipo I - Jato Medio,"Cor, urina",amarela,2020-01-06,Externo,Alta Administrativa,1974,F,GRUPO_3,0.0,False
359121,9F0062E1BD4A5603EBAF4593C7866332,2020-05-20,Urina Tipo I - Jato Medio,"Cor, urina",amarela,2020-05-20,Pronto Atendimento,Alta médica melhorado,1985,M,GRUPO_3,0.0,False
359122,4AF863B9C598125B9E27296C851AEFCB,2020-01-06,"Urina, Cultura - Jato Médio","Urina, Cultura - Jato Médio - Parcial",Parcial Negativo,2020-01-06,Externo,Alta Administrativa,1974,F,GRUPO_3,0.0,False
359123,4AF863B9C598125B9E27296C851AEFCB,2020-01-06,"Urina, Cultura - Jato Médio",Cultura de Urina,NEGATIVO,2020-01-06,Externo,Alta Administrativa,1974,F,GRUPO_3,0.0,False


In [ ]:
# Apaga linhas em que Resultado não é do tipo numérico

df_hsl_1.drop(df_hsl_1.loc[df_hsl_1['Tipo_resultado']==False].index, inplace=True)

In [ ]:
df_hsl_1.shape

(277641, 13)

In [ ]:
df_hsl_1[df_hsl_1['Tipo_resultado']==False]

,ID_PACIENTE,DT_COLETA,DE_EXAME,DE_ANALITO,DE_RESULTADO,DT_ATENDIMENTO,DE_TIPO_ATENDIMENTO,DE_DESFECHO,aa_nascimento,IC_SEXO,GRUPO,PERIODO_EXAMES,Tipo_resultado


In [ ]:
df_hsl_1['ID_PACIENTE'].nunique()

4405

#### **CONVERTENDO RESULTADOS DE STR PARA FLOAT**

In [ ]:
df_hsl_1['DE_RESULTADO'] = df_hsl_1['DE_RESULTADO'].astype(float) 

#### **NOVA SELEÇÃO MANUAL DE ATRIBUTOS**

In [ ]:
pivot_sirio = df_hsl_1.drop(columns=(['DE_EXAME', 
                                      'DT_ATENDIMENTO', 
                                      'DE_TIPO_ATENDIMENTO', 
                                      'DE_DESFECHO', 
                                      'PERIODO_EXAMES', 
                                      'Tipo_resultado']))

In [ ]:
pivot_sirio['ID_PACIENTE'].value_counts()

875B9333DA03891BE8AE5BC11EF8E982    1214
5E46317D5210A201322D0BFE7558A0EA    1098
AC1935CDA2C630ECDCCF6802E5B43D2F    1084
9A2FEF0E972CE7E63B4D268080E58085     956
30111D614617ACC12EA93EB3C24A1765     903
                                    ... 
4AAFAC84FD11DB3C084AB3E464069573       1
193715EA8E2C9E9C6B7050BA5845E419       1
5B1C222D15CE67BE56548939BC5EB062       1
0E43BFC81EB55DDB805A524E7601AC95       1
9361366DAD76154CD0E4F7B88545E5C9       1
Name: ID_PACIENTE, Length: 4405, dtype: int64

In [ ]:
pivot_sirio.head(3)

,ID_PACIENTE,DT_COLETA,DE_ANALITO,DE_RESULTADO,aa_nascimento,IC_SEXO,GRUPO
0,9DECD77C85AD8EF58C52F5D747546632,2020-06-10,17-Hidroxi Pregnenolona,44.0,1982,F,GRUPO_0
1,CE22DCFCE0BF5718B1781E4F01643661,2020-05-26,"Ácido ascorbico, plasma",0.8,1984,F,GRUPO_0
2,9A76D584342057BB53DA0E2032E45DE5,2020-10-30,"Ácido ascorbico, plasma",0.8,1993,M,GRUPO_0


In [ ]:
#pivot_sirio.to_csv("ANALISE_SIRIO_FINAL.csv", encoding="utf-8")

In [ ]:
pivot_sirio.shape

(277641, 7)

#### **GERANDO DATAFRAME SIRIO_APRENDIZADO**

In [ ]:
AL7 = pivot_sirio

# AL7 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/2021 dezembro Artigo/ANALISE_SIRIO_FINAL.csv', sep=',', index_col=0)

AL7.head(3)

,ID_PACIENTE,DT_COLETA,DE_ANALITO,DE_RESULTADO,aa_nascimento,IC_SEXO,GRUPO
0,9DECD77C85AD8EF58C52F5D747546632,2020-06-10,17-Hidroxi Pregnenolona,44.0,1982,F,GRUPO_0
1,CE22DCFCE0BF5718B1781E4F01643661,2020-05-26,"Ácido ascorbico, plasma",0.8,1984,F,GRUPO_0
2,9A76D584342057BB53DA0E2032E45DE5,2020-10-30,"Ácido ascorbico, plasma",0.8,1993,M,GRUPO_0


#### **FILTRO 8: AGRUPAMENTO DE EXAMES REPETIDOS PARA O MESMO PACIENTE**

In [ ]:
#Considera apenas o último exame no caso de repetidos.

AL7 = AL7.groupby(['ID_PACIENTE', 'GRUPO','aa_nascimento','IC_SEXO','DE_ANALITO']).agg({'DT_COLETA': ['max'], 'DE_RESULTADO' : ['last']}).reset_index()

AL7.head(3)

,ID_PACIENTE,GRUPO,aa_nascimento,IC_SEXO,DE_ANALITO,DT_COLETA,DE_RESULTADO
,,,,,,max,last
0,00017961865C4F766FDBB3CD8FE0BFB0,GRUPO_0,1967,M,ALT (TGP),2020-08-25,26.0
1,00017961865C4F766FDBB3CD8FE0BFB0,GRUPO_0,1967,M,AST (TGO),2020-08-25,24.0
2,00017961865C4F766FDBB3CD8FE0BFB0,GRUPO_0,1967,M,Basófilos,2020-08-25,40.0


In [ ]:
AL7.columns = ['ID_PACIENTE', 'GRUPO', 'Idade','Sexo', 'DE_ANALITO','DT_COLETA', 'DE_RESULTADO']

AL7.head(3)

,ID_PACIENTE,GRUPO,Idade,Sexo,DE_ANALITO,DT_COLETA,DE_RESULTADO
0,00017961865C4F766FDBB3CD8FE0BFB0,GRUPO_0,1967,M,ALT (TGP),2020-08-25,26.0
1,00017961865C4F766FDBB3CD8FE0BFB0,GRUPO_0,1967,M,AST (TGO),2020-08-25,24.0
2,00017961865C4F766FDBB3CD8FE0BFB0,GRUPO_0,1967,M,Basófilos,2020-08-25,40.0


In [ ]:
AL7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171829 entries, 0 to 171828
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   ID_PACIENTE   171829 non-null  object 
 1   GRUPO         171829 non-null  object 
 2   Idade         171829 non-null  object 
 3   Sexo          171829 non-null  object 
 4   DE_ANALITO    171829 non-null  object 
 5   DT_COLETA     171829 non-null  object 
 6   DE_RESULTADO  171829 non-null  float64
dtypes: float64(1), object(6)
memory usage: 9.2+ MB


In [ ]:
AL7.shape

(171829, 7)

In [ ]:
AL7['ID_PACIENTE'].nunique()

4405

In [ ]:
# Transformando Ano de nascimento em idade
# Erro porque pacientes com ano de nascimento = YYYY ou AAAA 
# Esses exemplos serão removidos nos próximos passos

#AL7['Idade'] = AL7['Idade'].astype(int)

#### **FILTRO 9: ELIMINANDO LINHAS COM DATA DE NASCIMENTO AAAA OU YYYY**

In [ ]:
AL7.drop(AL7.loc[AL7['Idade']=='AAAA'].index, inplace=True)
AL7.drop(AL7.loc[AL7['Idade']=='YYYY'].index, inplace=True)

In [ ]:
AL7['Idade'] = AL7['Idade'].astype(int)

In [ ]:
AL7['Idade'] = 2021 - AL7['Idade']

AL7.head(3)

,ID_PACIENTE,GRUPO,Idade,Sexo,DE_ANALITO,DT_COLETA,DE_RESULTADO
0,00017961865C4F766FDBB3CD8FE0BFB0,GRUPO_0,54,M,ALT (TGP),2020-08-25,26.0
1,00017961865C4F766FDBB3CD8FE0BFB0,GRUPO_0,54,M,AST (TGO),2020-08-25,24.0
2,00017961865C4F766FDBB3CD8FE0BFB0,GRUPO_0,54,M,Basófilos,2020-08-25,40.0


In [ ]:
AL7.shape

(168910, 7)

In [ ]:
AL7['ID_PACIENTE'].nunique()

4320

In [ ]:
#AL7.to_excel("ANALISE_SIRIO_FINAL_v2.xlsx")

#### **FILTRO 10: PIVOTAMENTO PARA OS EXAMES SE TORNAREM COLUNAS**

In [ ]:
sirio_aprendizado = AL7.pivot_table(index=['ID_PACIENTE','GRUPO','Idade','Sexo'], 
                                      
                                      values=['DE_RESULTADO'],

                                      columns=['DE_ANALITO'],

                                      aggfunc=[np.mean]).reset_index()

sirio_aprendizado.head(3)

ID_PACIENTE    GRUPO Idade Sexo  \
                                                                   
DE_ANALITO                                                         
0           00017961865C4F766FDBB3CD8FE0BFB0  GRUPO_0    54    M   
1           000F0BC139D2846DB86AA32B8F05B215  GRUPO_0    41    M   
2           0028785949D91BD93442838FC898E229  GRUPO_0    31    F   

                           mean                              \
                   DE_RESULTADO                               
DE_ANALITO 17-Alfa-Hidroxiprog. 17-Hidroxi Pregnenolona 183   
0                           NaN                     NaN NaN   
1                           NaN                     NaN NaN   
2                           NaN                     NaN NaN   

                                                      ...  \
                                                      ...   
DE_ANALITO 25OH-Vitamina D Total 5-Nucleotidase ACTH  ...   
0                            NaN            NaN  NaN  ...   
1                            NaN            NaN  NaN  ...   
2                            NaN            NaN  NaN  ...   

                                                                 \
                                                                  
DE_ANALITO Ácido ascorbico, plasma Ácido aspártico Ácido fólico   
0                              NaN             NaN          NaN   
1                              NaN             NaN          NaN   
2                              NaN             NaN          NaN   

                                                                  \
                                                                   
DE_ANALITO Ácido glutâmico Ácido metilmalonico, soro Ácido Úrico   
0                      NaN                       NaN         NaN   
1                      NaN                       NaN         7.8   
2                      NaN                       NaN         NaN   

                                                      \
                                                       
DE_ANALITO Ácido Úrico, urina Ácidos biliares totais   
0                         NaN                    NaN   
1                         NaN                    NaN   
2                         NaN                    NaN   

                                                             
                                                             
DE_ANALITO Ésteres de Carnitina (EC) Índice de Green & King  
0                                NaN                    NaN  
1                                NaN                    NaN  
2                                NaN                    NaN  

[3 rows x 502 columns]

In [ ]:
sirio_aprendizado.to_csv('pivot_table_V3.csv', sep='|',encoding='utf-8')
#analise_sirio_final.to_excel("ANALISE_SIRIO_FINAL_v3.xlsx")

In [ ]:
#nomes_exames = sirio_aprendizado

nomes_exames = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/2021 dezembro Artigo/Arquivos_v3/pivot_table_v3.csv', sep='|')

nomes_exames.head(10)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,2

,Unnamed: 0,ID_PACIENTE,GRUPO,Idade,Sexo,mean,mean.1,mean.2,mean.3,mean.4,...,mean.488,mean.489,mean.490,mean.491,mean.492,mean.493,mean.494,mean.495,mean.496,mean.497
0,NaN,NaN,NaN,NaN,NaN,DE_RESULTADO,DE_RESULTADO,DE_RESULTADO,DE_RESULTADO,DE_RESULTADO,...,DE_RESULTADO,DE_RESULTADO,DE_RESULTADO,DE_RESULTADO,DE_RESULTADO,DE_RESULTADO,DE_RESULTADO,DE_RESULTADO,DE_RESULTADO,DE_RESULTADO
1,DE_ANALITO,NaN,NaN,NaN,NaN,17-Alfa-Hidroxiprog.,17-Hidroxi Pregnenolona,183,25OH-Vitamina D Total,5-Nucleotidase,...,"Ácido ascorbico, plasma",Ácido aspártico,Ácido fólico,Ácido glutâmico,"Ácido metilmalonico, soro",Ácido Úrico,"Ácido Úrico, urina",Ácidos biliares totais,Ésteres de Carnitina (EC),Índice de Green & King
2,0,00017961865C4F766FDBB3CD8FE0BFB0,GRUPO_0,54.0,M,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,000F0BC139D2846DB86AA32B8F05B215,GRUPO_0,41.0,M,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,7.8,NaN,NaN,NaN,NaN
4,2,0028785949D91BD93442838FC898E229,GRUPO_0,31.0,F,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3,002B919CC409B11DE52FB212379BE2CB,GRUPO_0,41.0,F,NaN,NaN,NaN,31.0,NaN,...,NaN,NaN,19.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,4,003051C9B19101D1C10C5DC654384017,GRUPO_0,36.0,M,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,5,003F1F4C194763E4A00FD809AF5FA6AA,GRUPO_1,65.0,M,NaN,NaN,62.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,6,004688799FD293C3ABE0A07209FD8B75,GRUPO_3,68.0,M,NaN,NaN,NaN,29.0,NaN,...,NaN,NaN,NaN,NaN,NaN,5.7,NaN,NaN,NaN,NaN
9,7,0060E857BDD25FCC2D86F44CBB5D9FD9,GRUPO_0,22.0,M,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
nomes_exames = nomes_exames.T

In [ ]:
nomes_exames.head(10)

,0,1,2,3,4,5,6,7,8,9,...,4312,4313,4314,4315,4316,4317,4318,4319,4320,4321
Unnamed: 0,NaN,DE_ANALITO,0,1,2,3,4,5,6,7,...,4310,4311,4312,4313,4314,4315,4316,4317,4318,4319
ID_PACIENTE,NaN,NaN,00017961865C4F766FDBB3CD8FE0BFB0,000F0BC139D2846DB86AA32B8F05B215,0028785949D91BD93442838FC898E229,002B919CC409B11DE52FB212379BE2CB,003051C9B19101D1C10C5DC654384017,003F1F4C194763E4A00FD809AF5FA6AA,004688799FD293C3ABE0A07209FD8B75,0060E857BDD25FCC2D86F44CBB5D9FD9,...,FF4DF2541B0DCF41BCFBD3863C82F96D,FF6D3EC0F8DE67312BF8A327FB7FEFB4,FF7DABED9ADADB8808A3A0BFF9980BC7,FF9E422DCBA26A6C8E05CE8FA477B262,FFAADAF5DEC2C337DAC6CD9A7536122B,FFABB233208E1E66DDC025BC5CC2E4D2,FFB440876DC059A0359FEFBA1D6FB28C,FFEC3898BAA04751EB00C108270B8F7E,FFF5753408C98D5E0218931420B6AF85,FFFA1AB079C74F2EFEA6BA07E5B5DFC9
GRUPO,NaN,NaN,GRUPO_0,GRUPO_0,GRUPO_0,GRUPO_0,GRUPO_0,GRUPO_1,GRUPO_3,GRUPO_0,...,GRUPO_0,GRUPO_0,GRUPO_0,GRUPO_0,GRUPO_0,GRUPO_1,GRUPO_0,GRUPO_0,GRUPO_0,GRUPO_0
Idade,NaN,NaN,54.0,41.0,31.0,41.0,36.0,65.0,68.0,22.0,...,38.0,51.0,58.0,39.0,75.0,68.0,20.0,39.0,17.0,54.0
Sexo,NaN,NaN,M,M,F,F,M,M,M,M,...,F,F,M,F,F,M,M,M,F,F
mean,DE_RESULTADO,17-Alfa-Hidroxiprog.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean.1,DE_RESULTADO,17-Hidroxi Pregnenolona,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean.2,DE_RESULTADO,183,NaN,NaN,NaN,NaN,NaN,62.0,NaN,NaN,...,NaN,NaN,61.0,180.0,NaN,NaN,NaN,NaN,NaN,NaN
mean.3,DE_RESULTADO,25OH-Vitamina D Total,NaN,NaN,NaN,31.0,NaN,NaN,29.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean.4,DE_RESULTADO,5-Nucleotidase,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#nomes_exames = nomes_exames.reset_index()

In [ ]:
#nomes_exames.head(5)

In [ ]:
nomes = nomes_exames[1].values.tolist()

In [ ]:
nomes[0:10]

['DE_ANALITO',
 nan,
 nan,
 nan,
 nan,
 '17-Alfa-Hidroxiprog.',
 '17-Hidroxi Pregnenolona',
 '183',
 '25OH-Vitamina D Total',
 '5-Nucleotidase']

In [ ]:
nomes[0] = 'ID_PACIENTE'
nomes[1] = 'GRUPO'
nomes[2] = 'Idade'
nomes[3] = 'Sexo'
del nomes[4] # deleta valor nulo
nomes[0:10]

['ID_PACIENTE',
 'GRUPO',
 'Idade',
 'Sexo',
 '17-Alfa-Hidroxiprog.',
 '17-Hidroxi Pregnenolona',
 '183',
 '25OH-Vitamina D Total',
 '5-Nucleotidase',
 'ACTH']

#### **TABELA FINAL PARA MANIPULAÇÃO**

In [ ]:
sirio_aprendizado.columns = nomes

In [ ]:
sirio_aprendizado.head(5)

,ID_PACIENTE,GRUPO,Idade,Sexo,17-Alfa-Hidroxiprog.,17-Hidroxi Pregnenolona,183,25OH-Vitamina D Total,5-Nucleotidase,ACTH,...,"Ácido ascorbico, plasma",Ácido aspártico,Ácido fólico,Ácido glutâmico,"Ácido metilmalonico, soro",Ácido Úrico,"Ácido Úrico, urina",Ácidos biliares totais,Ésteres de Carnitina (EC),Índice de Green & King
0,00017961865C4F766FDBB3CD8FE0BFB0,GRUPO_0,54,M,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000F0BC139D2846DB86AA32B8F05B215,GRUPO_0,41,M,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,7.8,NaN,NaN,NaN,NaN
2,0028785949D91BD93442838FC898E229,GRUPO_0,31,F,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,002B919CC409B11DE52FB212379BE2CB,GRUPO_0,41,F,NaN,NaN,NaN,31.0,NaN,NaN,...,NaN,NaN,19.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,003051C9B19101D1C10C5DC654384017,GRUPO_0,36,M,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#sirio_aprendizado.to_excel("dados_pre_processado.xlsx")

In [ ]:
sirio_aprendizado.shape

(4320, 502)

#### **VERIFICANDO VALORES NULOS POR EXAME**

https://sigmoidal.ai/como-tratar-dados-ausentes-com-pandas/

In [ ]:
ausentes = sirio_aprendizado.isnull().sum()

ausentes = pd.DataFrame([ausentes])

ausentes = ausentes.drop(columns=['ID_PACIENTE', 'GRUPO'])

ausentes = ausentes.T

In [ ]:
ausentes

,0
Idade,0
Sexo,0
17-Alfa-Hidroxiprog.,4290
17-Hidroxi Pregnenolona,4317
183,3787
...,...
Ácido Úrico,3537
"Ácido Úrico, urina",4315
Ácidos biliares totais,4319
Ésteres de Carnitina (EC),4319


In [ ]:
ausentes['Nulos por cento'] = ''

ausentes.head(3)

,0,Nulos por cento
Idade,0,
Sexo,0,
17-Alfa-Hidroxiprog.,4290,


In [ ]:
for index, row in ausentes.iterrows():
        porcentagem = round((row[0]/4320)*100, 2)
        ausentes.loc[index,'Nulos por cento'] = porcentagem

In [ ]:
ausentes = ausentes.sort_values(by=[0], ascending=False)

In [ ]:
ausentes.head(3)

,0,Nulos por cento
Fator XI,4319,99.98
"Proteinase - 3, Anticorpos",4319,99.98
Rast - F2 - Leite de vaca,4319,99.98


In [ ]:
ausentes.reset_index(inplace=True, drop=False)

In [ ]:
ausentes.head()

,index,0,Nulos por cento
0,Fator XI,4319,99.98
1,"Proteinase - 3, Anticorpos",4319,99.98
2,Rast - F2 - Leite de vaca,4319,99.98
3,C1q,4319,99.98
4,Rast - F13 - Amendoim,4319,99.98


In [ ]:
ausentes.to_csv('ausentes.csv', sep='|', encoding='utf-8')

#### **FILTRO 11: ELIMINANDO EXAMES AUSENTES EM MAIS DE 50% DOS PACIENTES**

In [ ]:
ausentes.drop(ausentes.loc[ausentes['Nulos por cento']<50].index, inplace=True)

In [ ]:
ausentes

,index,0,Nulos por cento
0,Fator XI,4319,99.98
1,"Proteinase - 3, Anticorpos",4319,99.98
2,Rast - F2 - Leite de vaca,4319,99.98
3,C1q,4319,99.98
4,Rast - F13 - Amendoim,4319,99.98
...,...,...,...
466,DHL,2688,62.22
467,Gama-GT,2643,61.18
468,Cálculo p/não afrodescendente CKD-EPI,2560,59.26
469,"Dimeros D, quant",2332,53.98


In [ ]:
exames_eliminar = ausentes['index'].values.tolist()

In [ ]:
exames_eliminar[0:10]

['Fator XI',
 'Proteinase - 3, Anticorpos',
 'Rast - F2 - Leite de vaca',
 'C1q',
 'Rast - F13 - Amendoim',
 'Rast - F1 - Clara de ovo',
 'Enolase Neuronio-Específica',
 'Rast - E5 - epitélio de cão',
 'Bilirrubina,urina',
 'Epstein-Barr, IgG - Índice']

In [ ]:
sirio_aprendizado = sirio_aprendizado.drop(columns=(exames_eliminar))

In [ ]:
sirio_aprendizado.head(3)

,ID_PACIENTE,GRUPO,Idade,Sexo,ALT (TGP),AST (TGO),Basófilos,Basófilos (%),CHCM,Creatinina,...,Neutrófilos,Neutrófilos (%),Plaquetas,Potássio,Proteína C-Reativa,RDW,Sódio,Uréia,VCM,Volume plaquetário médio
0,00017961865C4F766FDBB3CD8FE0BFB0,GRUPO_0,54,M,26.0,24.0,40.0,0.6,34.5,1.02,...,5020.0,78.0,176000.0,4.0,0.11,13.1,138.0,35.0,86.0,9.8
1,000F0BC139D2846DB86AA32B8F05B215,GRUPO_0,41,M,NaN,NaN,20.0,0.4,33.2,1.04,...,1770.0,37.4,279000.0,4.3,NaN,14.0,142.0,33.0,83.2,10.0
2,0028785949D91BD93442838FC898E229,GRUPO_0,31,F,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Substituindo sexo feminino por 0 e sexo masculino por 1

In [ ]:
sirio_aprendizado['SEXO'] = ''

for index, row in sirio_aprendizado.iterrows():
        if row['Sexo'] == 'F':
            sirio_aprendizado.loc[index,'SEXO'] = 0
    
        else:
            sirio_aprendizado.loc[index,'SEXO'] = 1


In [ ]:
sirio_aprendizado['Sexo'].value_counts()

F    2241
M    2079
Name: Sexo, dtype: int64

In [ ]:
sirio_aprendizado['SEXO'].value_counts()

0    2241
1    2079
Name: SEXO, dtype: int64

In [ ]:
#Converto Sexo para int

sirio_aprendizado['SEXO'] = sirio_aprendizado['SEXO'].astype(int) 

In [ ]:
sirio_aprendizado.head(10)

,ID_PACIENTE,GRUPO,Idade,Sexo,ALT (TGP),AST (TGO),Basófilos,Basófilos (%),CHCM,Creatinina,...,Neutrófilos (%),Plaquetas,Potássio,Proteína C-Reativa,RDW,Sódio,Uréia,VCM,Volume plaquetário médio,SEXO
0,00017961865C4F766FDBB3CD8FE0BFB0,GRUPO_0,54,M,26.0,24.0,40.0,0.6,34.5,1.02,...,78.0,176000.0,4.0,0.11,13.1,138.0,35.0,86.0,9.8,1
1,000F0BC139D2846DB86AA32B8F05B215,GRUPO_0,41,M,NaN,NaN,20.0,0.4,33.2,1.04,...,37.4,279000.0,4.3,NaN,14.0,142.0,33.0,83.2,10.0,1
2,0028785949D91BD93442838FC898E229,GRUPO_0,31,F,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,002B919CC409B11DE52FB212379BE2CB,GRUPO_0,41,F,26.0,26.0,40.0,0.6,32.9,0.73,...,64.5,275000.0,NaN,0.12,13.3,NaN,30.0,88.3,11.2,0
4,003051C9B19101D1C10C5DC654384017,GRUPO_0,36,M,27.0,18.0,10.0,0.3,33.4,0.97,...,48.2,231000.0,4.2,0.08,14.4,139.0,16.0,83.7,10.4,1
5,003F1F4C194763E4A00FD809AF5FA6AA,GRUPO_1,65,M,79.0,76.0,10.0,0.2,33.4,0.95,...,75.6,212000.0,4.2,14.09,12.9,135.0,25.0,91.8,10.2,1
6,004688799FD293C3ABE0A07209FD8B75,GRUPO_3,68,M,31.0,21.0,30.0,0.4,32.5,2.07,...,68.8,215000.0,4.8,0.13,13.6,142.0,68.0,96.9,10.4,1
7,0060E857BDD25FCC2D86F44CBB5D9FD9,GRUPO_0,22,M,43.0,32.0,10.0,0.2,35.6,1.24,...,67.8,116000.0,4.1,0.65,14.6,139.0,22.0,77.7,10.9,1
8,007D8341B5ACFE2638807598AD434475,GRUPO_0,51,M,32.0,20.0,0.0,0.0,34.0,1.12,...,72.7,193000.0,3.8,0.95,12.9,138.0,29.0,83.6,10.7,1
9,008662E19D030205748A4B287F191B28,GRUPO_0,58,F,NaN,NaN,10.0,0.2,34.8,0.66,...,68.2,226000.0,4.2,6.47,11.9,136.0,12.0,90.3,9.6,0


In [ ]:
#Gerando arquivo csv

sirio_aprendizado.to_csv('sirio_aprendizado_v3.csv', sep='|', encoding='utf-8')

## **ANÁLISE DOS DADOS**

In [ ]:
sirio_aprendizado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4320 entries, 0 to 4319
Data columns (total 32 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID_PACIENTE               4320 non-null   object 
 1   GRUPO                     4320 non-null   object 
 2   Idade                     4320 non-null   int64  
 3   Sexo                      4320 non-null   object 
 4   ALT (TGP)                 2751 non-null   float64
 5   AST (TGO)                 2730 non-null   float64
 6   Basófilos                 3933 non-null   float64
 7   Basófilos (%)             3933 non-null   float64
 8   CHCM                      3931 non-null   float64
 9   Creatinina                3455 non-null   float64
 10  Eosinófilos               3933 non-null   float64
 11  Eosinófilos (%)           3933 non-null   float64
 12  Eritrócitos               3933 non-null   float64
 13  HCM                       3931 non-null   float64
 14  Hematócr

In [ ]:
sirio_aprendizado.shape

(4320, 32)

In [ ]:
sirio_aprendizado.head()

,ID_PACIENTE,GRUPO,Idade,Sexo,ALT (TGP),AST (TGO),Basófilos,Basófilos (%),CHCM,Creatinina,...,Neutrófilos (%),Plaquetas,Potássio,Proteína C-Reativa,RDW,Sódio,Uréia,VCM,Volume plaquetário médio,SEXO
0,00017961865C4F766FDBB3CD8FE0BFB0,GRUPO_0,54,M,26.0,24.0,40.0,0.6,34.5,1.02,...,78.0,176000.0,4.0,0.11,13.1,138.0,35.0,86.0,9.8,1
1,000F0BC139D2846DB86AA32B8F05B215,GRUPO_0,41,M,NaN,NaN,20.0,0.4,33.2,1.04,...,37.4,279000.0,4.3,NaN,14.0,142.0,33.0,83.2,10.0,1
2,0028785949D91BD93442838FC898E229,GRUPO_0,31,F,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,002B919CC409B11DE52FB212379BE2CB,GRUPO_0,41,F,26.0,26.0,40.0,0.6,32.9,0.73,...,64.5,275000.0,NaN,0.12,13.3,NaN,30.0,88.3,11.2,0
4,003051C9B19101D1C10C5DC654384017,GRUPO_0,36,M,27.0,18.0,10.0,0.3,33.4,0.97,...,48.2,231000.0,4.2,0.08,14.4,139.0,16.0,83.7,10.4,1


In [ ]:
#Quantidade de pacientes por grupo

G0 = sirio_aprendizado[sirio_aprendizado['GRUPO']=='GRUPO_0']
print('GRUPO_0: ', G0.shape)

G1 = sirio_aprendizado[sirio_aprendizado['GRUPO']=='GRUPO_1']
print('GRUPO_1: ', G1.shape)

G2 = sirio_aprendizado[sirio_aprendizado['GRUPO']=='GRUPO_2']
print('GRUPO_2: ', G2.shape)

G3 = sirio_aprendizado[sirio_aprendizado['GRUPO']=='GRUPO_3']
print('GRUPO_3: ', G3.shape)


GRUPO_0:  (3393, 32)
GRUPO_1:  (533, 32)
GRUPO_2:  (85, 32)
GRUPO_3:  (309, 32)


In [ ]:
# Valores ausentes por exames
mi = sirio_aprendizado.isnull().sum()

#mi = mi.sort()

mi

ID_PACIENTE                    0
GRUPO                          0
Idade                          0
Sexo                           0
ALT (TGP)                   1569
AST (TGO)                   1590
Basófilos                    387
Basófilos (%)                387
CHCM                         389
Creatinina                   865
Eosinófilos                  387
Eosinófilos (%)              387
Eritrócitos                  387
HCM                          389
Hematócrito                  386
Hemoglobina                  384
Leucócitos                   387
Linfócitos                   387
Linfócitos (%)               387
Monócitos                    387
Monócitos (%)                387
Neutrófilos                  387
Neutrófilos (%)              389
Plaquetas                    380
Potássio                    1441
Proteína C-Reativa          1266
RDW                          387
Sódio                       1501
Uréia                       1073
VCM                          387
Volume pla

In [ ]:
mi = pd.Series(mi)
#mi.index = X_train.columns
mi=mi.sort_values(ascending = False)
my_colors = ['r', 'g', 'b', 'k', 'y', 'm', 'c']
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
mi.plot(kind='bar', color=my_colors, figsize=(15,3))
plt.show()

In [ ]:
#ausentes por grupo

ausentes_G0 = G0.isnull().sum()
ausentes_G0 = pd.DataFrame([ausentes_G0])
ausentes_G0 = ausentes_G0.T
ausentes_G0.reset_index(inplace=True, drop=False)
ausentes_G0.columns = [['exame','G0']]

ausentes_G1 = G1.isnull().sum()
ausentes_G1 = pd.DataFrame([ausentes_G1])
ausentes_G1 = ausentes_G1.T
ausentes_G1.reset_index(inplace=True, drop=False)
ausentes_G1.columns = [['exame','G1']]

ausentes_G2 = G2.isnull().sum()
ausentes_G2 = pd.DataFrame([ausentes_G2])
ausentes_G2 = ausentes_G2.T
ausentes_G2.columns = ['G2']
ausentes_G2.reset_index(inplace=True, drop=False)
ausentes_G2.columns = [['exame','G2']]

ausentes_G3 = G3.isnull().sum()
ausentes_G3 = pd.DataFrame([ausentes_G3])
ausentes_G3 = ausentes_G3.T
ausentes_G3.columns = ['G3']
ausentes_G3.reset_index(inplace=True, drop=False)
ausentes_G3.columns = [['exame','G3']]

df_ausentes = ausentes_G0.merge(ausentes_G1)
df_ausentes = df_ausentes.merge(ausentes_G2)
df_ausentes = df_ausentes.merge(ausentes_G3)
df_ausentes

,exame,G0,G1,G2,G3
0,ID_PACIENTE,0,0,0,0
1,GRUPO,0,0,0,0
2,Idade,0,0,0,0
3,Sexo,0,0,0,0
4,ALT (TGP),1304,150,22,93
5,AST (TGO),1324,150,22,94
6,Basófilos,335,26,4,22
7,Basófilos (%),335,26,4,22
8,CHCM,335,27,5,22
9,Creatinina,745,66,11,43


#### **DESCRIBE**

In [ ]:
des_G0 = G0.describe().T
des_G1 = G1.describe().T
des_G2 = G2.describe().T
des_G3 = G3.describe().T

In [ ]:
des_G0.head(3)

,count,mean,std,min,25%,50%,75%,max
Idade,3393.0,43.231064,13.925899,2.0,34.0,42.0,52.0,89.0
ALT (TGP),2089.0,29.095740,25.924910,5.0,15.0,22.0,34.0,343.0
AST (TGO),2069.0,25.781054,15.958387,8.0,18.0,22.0,29.0,242.0


In [ ]:
describe_G0 = des_G0.drop(columns=['count','min','25%','50%','75%','max'])
describe_G0.reset_index(inplace=True, drop=False)
describe_G0.columns = ['Exame', 'mean_G0', 'std_G0']

describe_G1 = des_G1.drop(columns=['count','min','25%','50%','75%','max'])
describe_G1.reset_index(inplace=True, drop=False)
describe_G1.columns = ['Exame', 'mean_G1', 'std_G1']

describe_G2 = des_G2.drop(columns=['count','min','25%','50%','75%','max'])
describe_G2.reset_index(inplace=True, drop=False)
describe_G2.columns = ['Exame', 'mean_G2', 'std_G2']

describe_G3 = des_G3.drop(columns=['count','min','25%','50%','75%','max'])
describe_G3.reset_index(inplace=True, drop=False)
describe_G3.columns = ['Exame', 'mean_G3', 'std_G3']

In [ ]:
describe_G0.head(3)

,Exame,mean_G0,std_G0
0,Idade,43.231064,13.925899
1,ALT (TGP),29.095740,25.924910
2,AST (TGO),25.781054,15.958387


In [ ]:
print(describe_G0.shape)
print(describe_G1.shape)
print(describe_G2.shape)
print(describe_G3.shape)

(29, 3)
(29, 3)
(29, 3)
(29, 3)


In [ ]:
#Juntando o describe de cada grupo em um único dataframe
df_describe = describe_G0.merge(describe_G1, on = ["Exame"], how = "left")
df_describe = df_describe.merge(describe_G2, on=['Exame'], how = 'left')
df_describe = df_describe.merge(describe_G3, on=['Exame'], how = 'left')
df_describe.head(3)

,Exame,mean_G0,std_G0,mean_G1,std_G1,mean_G2,std_G2,mean_G3,std_G3
0,Idade,43.231064,13.925899,53.330206,14.803975,63.094118,18.491712,62.472492,14.687327
1,ALT (TGP),29.095740,25.924910,37.796345,36.588704,40.523810,42.169725,34.819444,27.732271
2,AST (TGO),25.781054,15.958387,29.506527,18.500866,28.380952,15.215048,29.139535,20.069717


In [ ]:
decimals = 2    
df_describe[['mean_G0', 'std_G0', 'mean_G1', 'std_G1', 'mean_G2', 'std_G2', 'mean_G3', 'std_G3']] = df_describe[['mean_G0', 'std_G0', 'mean_G1', 'std_G1', 'mean_G2', 'std_G2', 'mean_G3', 'std_G3']].apply(lambda x: round(x, decimals))
df_describe

,Exame,mean_G0,std_G0,mean_G1,std_G1,mean_G2,std_G2,mean_G3,std_G3
0,Idade,43.23,13.93,53.33,14.80,63.09,18.49,62.47,14.69
1,ALT (TGP),29.10,25.92,37.80,36.59,40.52,42.17,34.82,27.73
2,AST (TGO),25.78,15.96,29.51,18.50,28.38,15.22,29.14,20.07
3,Basófilos,30.82,20.65,29.47,21.20,36.42,47.55,32.09,25.21
4,Basófilos (%),0.51,0.31,0.47,0.31,0.50,0.43,0.49,0.37
5,CHCM,33.61,1.09,33.61,1.15,33.34,1.13,33.25,1.27
6,Creatinina,0.90,0.26,0.95,0.26,1.07,0.50,1.18,0.94
7,Eosinófilos,115.81,137.90,100.34,116.22,97.90,99.91,116.48,123.91
8,Eosinófilos (%),1.84,1.81,1.61,1.73,1.36,1.28,1.94,2.32
9,Eritrócitos,4.78,0.53,4.75,0.56,4.64,0.62,4.55,0.68


In [ ]:
df_describe.to_csv('df_describe.csv', sep='|', encoding='utf-8')

### **BOXPLOT**

In [ ]:
plt.title("Idade", size=14)
plt.gcf().set_size_inches(15, 3) # alterar tamanho
plt.xticks(range(0, 100, 10)) # mudar escala do eixo X
ax = sns.boxplot(x="Idade", y="GRUPO", data=sirio_aprendizado)

In [ ]:
plt.title("ALT (TGP)", size=14)
plt.gcf().set_size_inches(15, 3) # alterar tamanho
plt.xticks(range(0, 420, 15)) # mudar escala do eixo X
ax = sns.boxplot(x="ALT (TGP)", y="GRUPO", data=sirio_aprendizado)

In [ ]:
plt.title("AST (TGO)", size=14)
plt.gcf().set_size_inches(15, 3) # alterar tamanho
plt.xticks(range(0, 250, 10)) # mudar escala do eixo X
ax = sns.boxplot(x="AST (TGO)", y="GRUPO", data=sirio_aprendizado)

In [ ]:
plt.title("Basófilos", size=14)
plt.gcf().set_size_inches(15, 3) # alterar tamanho
plt.xticks(range(0, 420, 15)) # mudar escala do eixo X
ax = sns.boxplot(x="Basófilos", y="GRUPO", data=sirio_aprendizado)

In [ ]:
plt.title("Basófilos (%)", size=14)
plt.gcf().set_size_inches(15, 3) # alterar tamanho
plt.xticks(range(0, 5, 1)) # mudar escala do eixo X
ax = sns.boxplot(x="Basófilos (%)", y="GRUPO", data=sirio_aprendizado)

In [ ]:
plt.title("CHCM", size=14)
plt.gcf().set_size_inches(15, 3) # alterar tamanho
plt.xticks(range(0, 38, 1)) # mudar escala do eixo X
ax = sns.boxplot(x="CHCM", y="GRUPO", data=sirio_aprendizado)

In [ ]:
plt.title("Creatinina", size=14)
plt.gcf().set_size_inches(15, 3) # alterar tamanho
plt.xticks(range(0, 10, 1)) # mudar escala do eixo X
ax = sns.boxplot(x="Creatinina", y="GRUPO", data=sirio_aprendizado)

In [ ]:
plt.title("Eosinófilos", size=14)
plt.gcf().set_size_inches(15, 3) # alterar tamanho
plt.xticks(range(0, 3200, 100)) # mudar escala do eixo X
ax = sns.boxplot(x="Eosinófilos", y="GRUPO", data=sirio_aprendizado)

In [ ]:
plt.title("Eosinófilos (%)", size=14)
plt.gcf().set_size_inches(15, 3) # alterar tamanho
plt.xticks(range(0, 25, 1)) # mudar escala do eixo X
ax = sns.boxplot(x="Eosinófilos (%)", y="GRUPO", data=sirio_aprendizado)

In [ ]:
plt.title("Eritrócitos", size=14)
plt.gcf().set_size_inches(15, 3) # alterar tamanho
plt.xticks(range(0, 8, 1)) # mudar escala do eixo X
ax = sns.boxplot(x="Eritrócitos", y="GRUPO", data=sirio_aprendizado)

In [ ]:
plt.title("HCM", size=14)
plt.gcf().set_size_inches(15, 3) # alterar tamanho
plt.xticks(range(0, 41, 1)) # mudar escala do eixo X
ax = sns.boxplot(x="HCM", y="GRUPO", data=sirio_aprendizado)

In [ ]:
plt.title("Hematócrito", size=14)
plt.gcf().set_size_inches(15, 3) # alterar tamanho
plt.xticks(range(0, 65, 1)) # mudar escala do eixo X
ax = sns.boxplot(x="Hematócrito", y="GRUPO", data=sirio_aprendizado)

In [ ]:
plt.title("Hemoglobina", size=14)
plt.gcf().set_size_inches(15, 3) # alterar tamanho
plt.xticks(range(0, 25, 1)) # mudar escala do eixo X
ax = sns.boxplot(x="Hemoglobina", y="GRUPO", data=sirio_aprendizado)

In [ ]:
plt.title("Leucócitos", size=14)
plt.gcf().set_size_inches(15, 3) # alterar tamanho
plt.xticks(range(0, 120000, 10000)) # mudar escala do eixo X
ax = sns.boxplot(x="Leucócitos", y="GRUPO", data=sirio_aprendizado)

In [ ]:
plt.title("Linfócitos", size=14)
plt.gcf().set_size_inches(15, 3) # alterar tamanho
plt.xticks(range(0, 120000, 10000)) # mudar escala do eixo X
ax = sns.boxplot(x="Linfócitos", y="GRUPO", data=sirio_aprendizado)

In [ ]:
plt.title("Linfócitos (%)", size=14)
plt.gcf().set_size_inches(15, 3) # alterar tamanho
plt.xticks(range(0, 100, 5)) # mudar escala do eixo X
ax = sns.boxplot(x="Linfócitos (%)", y="GRUPO", data=sirio_aprendizado)

In [ ]:
plt.title("Monócitos", size=14)
plt.gcf().set_size_inches(15, 3) # alterar tamanho
plt.xticks(range(0, 4000, 500)) # mudar escala do eixo X
ax = sns.boxplot(x="Monócitos", y="GRUPO", data=sirio_aprendizado)

In [ ]:
plt.title("Monócitos (%)", size=14)
plt.gcf().set_size_inches(15, 3) # alterar tamanho
plt.xticks(range(0, 100, 1)) # mudar escala do eixo X
ax = sns.boxplot(x="Monócitos (%)", y="GRUPO", data=sirio_aprendizado)

In [ ]:
plt.title("Neutrófilos", size=14)
plt.gcf().set_size_inches(15, 3) # alterar tamanho
plt.xticks(range(0, 30000, 2500)) # mudar escala do eixo X
ax = sns.boxplot(x="Neutrófilos", y="GRUPO", data=sirio_aprendizado)

In [ ]:
plt.title("Neutrófilos (%)", size=14)
plt.gcf().set_size_inches(15, 3) # alterar tamanho
plt.xticks(range(0, 100, 10)) # mudar escala do eixo X
ax = sns.boxplot(x="Neutrófilos (%)", y="GRUPO", data=sirio_aprendizado)

In [ ]:
plt.title("Plaquetas", size=14)
plt.gcf().set_size_inches(15, 3) # alterar tamanho
plt.xticks(range(0, 750000, 50000)) # mudar escala do eixo X
ax = sns.boxplot(x="Plaquetas", y="GRUPO", data=sirio_aprendizado)

In [ ]:
plt.title("Potássio", size=14)
plt.gcf().set_size_inches(15, 3) # alterar tamanho
plt.xticks(range(0, 10, 1)) # mudar escala do eixo X
ax = sns.boxplot(x="Potássio", y="GRUPO", data=sirio_aprendizado)

In [ ]:
plt.title("Proteína C-Reativa", size=14)
plt.gcf().set_size_inches(15, 3) # alterar tamanho
plt.xticks(range(0, 100, 1)) # mudar escala do eixo X
ax = sns.boxplot(x="Proteína C-Reativa", y="GRUPO", data=sirio_aprendizado)

In [ ]:
plt.title("RDW", size=14)
plt.gcf().set_size_inches(15, 3) # alterar tamanho
plt.xticks(range(0, 30, 1)) # mudar escala do eixo X
ax = sns.boxplot(x="RDW", y="GRUPO", data=sirio_aprendizado)

In [ ]:
plt.title("Sódio", size=14)
plt.gcf().set_size_inches(15, 3) # alterar tamanho
plt.xticks(range(0, 170, 10)) # mudar escala do eixo X
ax = sns.boxplot(x="Sódio", y="GRUPO", data=sirio_aprendizado)

In [ ]:
plt.title("Uréia", size=14)
plt.gcf().set_size_inches(15, 3) # alterar tamanho
plt.xticks(range(0, 300, 10)) # mudar escala do eixo X
ax = sns.boxplot(x="Uréia", y="GRUPO", data=sirio_aprendizado)

In [ ]:
plt.title("VCM", size=14)
plt.gcf().set_size_inches(15, 3) # alterar tamanho
plt.xticks(range(0, 120, 10)) # mudar escala do eixo X
ax = sns.boxplot(x="VCM", y="GRUPO", data=sirio_aprendizado)

In [ ]:
plt.title("Volume plaquetário médio", size=14)
plt.gcf().set_size_inches(15, 3) # alterar tamanho
plt.xticks(range(0, 15, 1)) # mudar escala do eixo X
ax = sns.boxplot(x="Volume plaquetário médio", y="GRUPO", data=sirio_aprendizado)

In [ ]:
#! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [ ]:
#pandas_profiling.ProfileReport(sirio)

In [ ]:
#profile = sirio_aprendizado.profile_report(title="RELATORIO")
#profile.to_file(output_file="RELATORIO.html")